In [4]:
! git clone https://github.com/rami0205/RAMiT.git

fatal: destination path 'RAMiT' already exists and is not an empty directory.


## FFAT + RAMiT

In [5]:
# # RAMiT (Reciprocal Attention Mixing Transformer)

# from pdb import set_trace as stx
# import numbers
# from saicinpainting.training.modules.ffc0 import FFC_BN_ACT
# from skimage import io
# from skimage.metrics import peak_signal_noise_ratio as psnr





# from torch import nn, Tensor
# from einops import rearrange
# import torch.nn.functional as F
# import torch
# from torchvision.transforms import functional as TF
# import math
# from RAMiT.my_model.common.mean_std import mean_std
# from RAMiT.utils.etc_utils import denormalize
# from timm.models.fx_features import register_notrace_function
# from timm.models.layers import trunc_normal_, to_2tuple
# from torch.nn.modules.batchnorm import _BatchNorm
# def make_model(args, opts, pe):
#     model = RAMiT(target_mode=args.target_mode,
#                   img_norm=args.img_norm,
#                   in_chans=opts['in_chans'],
#                   dim = opts['dim'],
#                   depths = opts['depths'],
#                   num_heads = opts['num_heads'],
#                   head_dim = opts['head_dim'],
#                   chsa_head_ratio = opts['chsa_head_ratio'],
#                   window_size = opts['window_size'],
#                   hidden_ratio = opts['hidden_ratio'],
#                   qkv_bias = opts['qkv_bias'],
#                   mv_ver = opts['mv_ver'],
#                   exp_factor=opts['exp_factor'],
#                   expand_groups=opts['expand_groups'],
#                   act_layer = opts['act_layer'],
#                   norm_layer = opts['norm_layer'],
#                   tail_mv = opts['tail_mv'],
#                   attn_drop = opts['attn_drop'],
#                   proj_drop = opts['proj_drop'],
#                   drop_path = opts['drop_path'],
#                   helper = opts['helper'],
#                   mv_act = opts['mv_act']
#                  )
#     num_params = sum([p.numel() for n,p in model.named_parameters()])
#     print(format(num_params, ','))
#     return model





































# class ShallowModule(nn.Module):
#     def __init__(self, in_chans, out_chans, kernel_size=3, stride=1):
#         super(ShallowModule, self).__init__()

#         self.in_chans = in_chans
#         self.out_chans = out_chans
#         self.kernel_size = kernel_size
#         self.stride = stride

#         self.conv = nn.Conv2d(in_chans, out_chans, kernel_size, stride, kernel_size//2)

#     def forward(self, x):
#         return self.conv(x)

#     def flops(self, resolutions):
#         return resolutions[0]*resolutions[1] * self.kernel_size*self.kernel_size * self.in_chans * self.out_chans

# class QKVProjection(nn.Module):
#     def __init__(self, dim, num_head, qkv_bias=True):
#         super(QKVProjection, self).__init__()
#         self.dim = dim
#         self.num_head = num_head

#         self.qkv = nn.Conv2d(dim, 3*dim, 1, bias=qkv_bias)

#     def forward(self, x):
#         B, C, H, W = x.size()
#         qkv = self.qkv(x)
#         qkv = rearrange(qkv, 'b (l c) h w -> b l c h w', l=self.num_head)
#         return qkv

#     def flops(self, resolutions):
#         return resolutions[0]*resolutions[1] * 1*1 * self.dim * 3*self.dim

# def get_relative_position_index(win_h, win_w):
#     # get pair-wise relative position index for each token inside the window
#     coords = torch.stack(torch.meshgrid([torch.arange(win_h), torch.arange(win_w)]))  # 2, Wh, Ww
#     coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
#     relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww (xaxis matrix & yaxis matrix)
#     relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
#     relative_coords[:, :, 0] += win_h - 1  # shift to start from 0
#     relative_coords[:, :, 1] += win_w - 1
#     relative_coords[:, :, 0] *= 2 * win_w - 1
#     return relative_coords.sum(-1)  # Wh*Ww, Wh*Ww

# class SpatialSelfAttention(nn.Module):
#     def __init__(self, dim, num_head, total_head, window_size=8, shift=0, attn_drop=0.0, proj_drop=0.0, helper=True):
#         super(SpatialSelfAttention, self).__init__()
#         self.dim = dim
#         self.num_head = num_head
#         self.total_head = total_head
#         self.window_size = window_size
#         self.window_area = window_size**2
#         self.shift = shift
#         self.helper = helper

#         self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_head, 1, 1))), requires_grad=True)

#         # define a parameter table of relative position bias, shape: 2*Wh-1 * 2*Ww-1, nH
#         self.relative_position_bias_table = nn.Parameter(torch.zeros((2 * window_size - 1) * (2 * window_size - 1), num_head))

#         # get pair-wise relative position index for each token inside the window
#         self.register_buffer("relative_position_index", get_relative_position_index(window_size, window_size))

#         self.attn_drop = nn.Dropout(attn_drop)

#         self.proj = nn.Conv2d(dim*num_head, dim*num_head, 1)
#         self.proj_drop = nn.Dropout(proj_drop)

#     def _get_rel_pos_bias(self) -> torch.Tensor:
#         relative_position_bias = self.relative_position_bias_table[
#             self.relative_position_index.view(-1)].view(self.window_area, self.window_area, -1)  # Wh*Ww,Wh*Ww,nH
#         relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
#         return relative_position_bias.unsqueeze(0)

#     def forward(self, qkv, ch=None):
#         B, L, C, H, W = qkv.size()
#         # window shift
#         if self.shift > 0:
#             qkv = torch.roll(qkv, shifts=(-self.shift, -self.shift), dims=(-2,-1))

#         # window partition
#         q,k,v = rearrange(qkv, 'b l c (h wh) (w ww) -> (b h w) l (wh ww) c',
#                           wh=self.window_size, ww=self.window_size).chunk(3, dim=-1) # [B_, L1, hw, C/L] respectively
#         if ch is not None and self.helper: # [B, C, H, W]
#             if self.shift > 0:
#                 ch = torch.roll(ch, shifts=(-self.shift, -self.shift), dims=(-2,-1))
#             ch = rearrange(ch, 'b (l c) (h wh) (w ww) -> (b h w) l (wh ww) c',
#                            l=self.total_head-self.num_head, wh=self.window_size, ww=self.window_size) # [B_, L1, hw, C/L]
#             ch = torch.mean(ch, dim=1, keepdim=True) # head squeeze [B_, 1, hw, C/L]
#             v = v*ch # [B_, L1, hw, C/L]

#         attn = F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(2,-1) # [B_, L1, hw, hw]
#         logit_scale = torch.clamp(self.logit_scale, max=math.log(1. / 0.01)).exp()
#         attn = attn * logit_scale

#         attn = attn + self._get_rel_pos_bias()
#         attn = self.attn_drop(F.softmax(attn, dim=-1))

#         x = attn @ v # [B_, L1, hw, C/L]

#         # window unpartition + head merge
#         x = window_unpartition(x, (H,W), self.window_size) # [B, L1*C/L, H, W]
#         x = self.proj_drop(self.proj(x))

#         # window reverse shift
#         if self.shift > 0:
#             x = torch.roll(x, shifts=(self.shift, self.shift), dims=(-2,-1))

#         return x

#     def flops(self, resolutions):
#         H,W = resolutions
#         num_wins = H//self.window_size * W//self.window_size
#         flops = self.num_head * H*W * self.dim if self.helper else 0 # v = v*ch
#         flops += num_wins * self.num_head * self.window_area * self.dim * self.window_area # attn = Q@K^T
#         flops += num_wins * self.num_head * self.window_area * self.window_area * self.dim # attn@V
#         flops += H*W * 1*1 * self.num_head*self.dim * self.num_head*self.dim # self.proj
#         return flops

# @register_notrace_function
# def window_unpartition(x, resolutions, window_size):
#     return rearrange(x, '(b h w) l (wh ww) c -> b (l c) (h wh) (w ww)',
#                      h=resolutions[0]//window_size, w=resolutions[1]//window_size, wh=window_size)

# class ChannelSelfAttention(nn.Module):
#     def __init__(self, dim, num_head, total_head, attn_drop=0.0, proj_drop=0.0, helper=True):
#         super(ChannelSelfAttention, self).__init__()
#         self.dim = dim
#         self.num_head = num_head
#         self.total_head = total_head
#         self.helper = helper

#         self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_head, 1, 1))), requires_grad=True)

#         self.attn_drop = nn.Dropout(attn_drop)

#         self.proj = nn.Conv2d(dim*num_head, dim*num_head, 1)
#         self.proj_drop = nn.Dropout(proj_drop)

#     def forward(self, qkv, sp=None):
#         B, L, C, H, W = qkv.size()

#         q,k,v = rearrange(qkv, 'b l c h w -> b l c (h w)').chunk(3, dim=-2) # [B, L2, C/L, HW]
#         if sp is not None and self.helper:
#             sp = torch.mean(sp, dim=1, keepdim=True) # channel squeeze # [B, 1, H, W]
#             sp = rearrange(sp, 'b (l c) h w -> b l c (h w)', l=1) # [B, 1, 1, HW]
#             v = v*sp # [B, L2, C/L, HW]

#         attn = F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(2,-1) # [B, L2, C/L, C/L]
#         logit_scale = torch.clamp(self.logit_scale, max=math.log(1. / 0.01)).exp()
#         attn = attn * logit_scale

#         attn = F.softmax(attn, dim=-1)
#         attn = self.attn_drop(attn)

#         x = attn @ v # [B, L2, C/L, HW]

#         # head merge
#         x = rearrange(x, 'b l c (h w) -> b (l c) h w', h=H) # [B, L2*C/L, H, W]
#         x = self.proj_drop(self.proj(x)) # [B, L2*C/L, H, W]

#         return x

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = self.num_head * self.dim * H*W if self.helper else 0 # v = v*sp
#         flops += self.num_head * self.dim * H*W * self.dim # attn = Q@K^T
#         flops += self.num_head * self.dim * self.dim * H*W # attn@V
#         flops += H*W * 1*1 * self.num_head*self.dim * self.num_head*self.dim # self.proj
#         return flops

# class ReshapeLayerNorm(nn.Module):
#     def __init__(self, dim, norm_layer=nn.LayerNorm):
#         super(ReshapeLayerNorm, self).__init__()

#         self.dim = dim
#         self.norm = norm_layer(dim)

#     def forward(self, x):
#         B, C, H, W = x.size()
#         x = rearrange(x, 'b c h w -> b (h w) c')
#         x = self.norm(x)
#         x = rearrange(x, 'b (h w) c -> b c h w', h=H)
#         return x

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = 0
#         flops += H*W * self.dim
#         return flops

# class MobiVari1(nn.Module): # MobileNet v1 Variants
#     def __init__(self, dim, kernel_size, stride, act=nn.LeakyReLU, out_dim=None):
#         super(MobiVari1, self).__init__()
#         self.dim = dim
#         self.kernel_size = kernel_size
#         self.out_dim = out_dim or dim

#         self.dw_conv = nn.Conv2d(dim, dim, kernel_size, stride, kernel_size//2, groups=dim)
#         self.pw_conv = nn.Conv2d(dim, self.out_dim, 1, 1, 0)
#         self.act = act()

#     def forward(self, x):
#         out = self.act(self.pw_conv(self.act(self.dw_conv(x))+x))
#         return out + x if self.dim==self.out_dim else out

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = H*W * self.kernel_size*self.kernel_size * self.dim  +  H*W * 1*1 * self.dim * self.out_dim # self.dw_conv + self.pw_conv
#         return flops

# class MobiVari2(MobiVari1): # MobileNet v2 Variants
#     def __init__(self, dim, kernel_size, stride, act=nn.LeakyReLU, out_dim=None, exp_factor=1.2, expand_groups=4):
#         super(MobiVari2, self).__init__(dim, kernel_size, stride, act, out_dim)
#         self.expand_groups = expand_groups
#         expand_dim = int(dim*exp_factor)
#         expand_dim = expand_dim+(expand_groups-expand_dim%expand_groups)
#         self.expand_dim = expand_dim

#         self.exp_conv = nn.Conv2d(dim, self.expand_dim, 1, 1, 0, groups=expand_groups)
#         self.dw_conv = nn.Conv2d(expand_dim, expand_dim, kernel_size, stride, kernel_size//2, groups=expand_dim)
#         self.pw_conv = nn.Conv2d(expand_dim, self.out_dim, 1, 1, 0)

#     def forward(self, x):
#         x1 = self.act(self.exp_conv(x))
#         out = self.pw_conv(self.act(self.dw_conv(x1)+x1))
#         return out + x if self.dim==self.out_dim else out

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = H*W * 1*1 * (self.dim//self.expand_groups) * self.expand_dim # self.exp_conv
#         flops += H*W * self.kernel_size*self.kernel_size * self.expand_dim # self.dw_conv
#         flops += H*W * 1*1 * self.expand_dim * self.out_dim # self.pw_conv
#         return flops

# class FeedForward(nn.Module):
#     def __init__(self, dim, hidden_ratio, act_layer=nn.GELU, bias=True, drop=0.0):
#         super(FeedForward, self).__init__()

#         self.dim = dim
#         self.hidden_ratio = hidden_ratio

#         self.hidden = nn.Conv2d(dim, int(dim*hidden_ratio), 1, bias=bias)
#         self.drop1 = nn.Dropout(drop)
#         self.out = nn.Conv2d(int(dim*hidden_ratio), dim, 1, bias=bias)
#         self.drop2 = nn.Dropout(drop)
#         self.act = act_layer()

#     def forward(self, x):
#         return self.drop2(self.out(self.drop1(self.act(self.hidden(x)))))

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = 2 * H*W * 1*1 * self.dim * self.dim*self.hidden_ratio # self.hidden + self.out
#         return flops

# class NoLayer(nn.Identity):
#     def __init__(self):
#         super(NoLayer, self).__init__()
#     def flops(self, resolutions):
#         return 0
#     def forward(self, x, **kwargs):
#         return x.flatten(1,2)

# class DRAMiTransformer(nn.Module): # Reciprocal Attention Transformer Block
#     def __init__(self, dim, num_head, chsa_head_ratio, FFAT, window_size=8, shift=0, head_dim=None, qkv_bias=True, mv_ver=1,
#                  hidden_ratio=2.0, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True,
#                  mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4,
#                  num_blocks = [4,6,6,8], heads = [1,2,4,8], ffn_expansion_factor = 2.66, bias = False, LayerNorm_type = 'WithBias'):
#         super(DRAMiTransformer, self).__init__()

#         self.dim = dim
#         self.num_head = num_head
#         self.window_size = window_size
#         self.chsa_head = int(num_head*chsa_head_ratio)
#         self.shift = shift
#         self.helper = helper
#         self.FFAT = FFAT

#         self.qkv_proj = QKVProjection(dim, num_head, qkv_bias=qkv_bias)
#         self.sp_attn = SpatialSelfAttention(dim//num_head, num_head-self.chsa_head, num_head,
#                                             window_size, shift, attn_drop, proj_drop, helper) if num_head-self.chsa_head != 0 else NoLayer()
#         self.ch_attn = ChannelSelfAttention(dim//num_head, self.chsa_head, num_head, attn_drop, proj_drop, helper) if self.chsa_head != 0 else NoLayer()


#         #FFAT
#         if self.FFAT:
#           self.encoder_level1 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])
#         #FFAT


#         if mv_ver==3:
#             self.mobivari = MobiVari1(dim, 3, 1, act=mv_act)
#         elif mv_ver==2:
#             self.mobivari = MobiVari2(dim, 3, 1, act=mv_act, out_dim=None, exp_factor=exp_factor, expand_groups=expand_groups)
#             # NAFBlock---------------------------------------------
#         elif mv_ver==1:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------

#         self.norm1 = norm_layer(dim)

#         self.ffn = FeedForward(dim, hidden_ratio, act_layer=act_layer)
#         self.norm2 = norm_layer(dim)

#         self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

#     def forward(self, x, sp_=None, ch_=None):
#         B, C, H, W = x.size()

#         # QKV projection + head split
#         qkv = self.qkv_proj(x) # [B, L, C, H, W]

#         # SP-SA / CH-SA
#         sp = self.sp_attn(qkv[:,:self.num_head-self.chsa_head], ch=ch_) # [B, L1*C/L, H, W]
#         ch = self.ch_attn(qkv[:,self.num_head-self.chsa_head:], sp=sp_) # [B, L2*C/L, H, W]


#         #FFAT
#         if self.FFAT:
#           FFAT_block = self.encoder_level1(torch.cat([sp, ch], dim=1))
#           attn0 = self.mobivari(FFAT_block)
#         #FFAT


#         else:
#           attn0 = self.mobivari(torch.cat([sp, ch], dim=1)) # merge [B, C, H, W]
#         attn = self.drop_path(self.norm1(attn0)) + x # LN, skip connection [B, C, H, W]

#         # FFN
#         out = self.drop_path(self.norm2(self.ffn(attn))) + attn # FFN, LN, skip connection [B, C, H, W]

#         return out, sp, ch, attn0

#     def flops(self, resolutions):
#         flops = self.qkv_proj.flops(resolutions)
#         flops += self.sp_attn.flops(resolutions)
#         flops += self.ch_attn.flops(resolutions)
#         flops += self.mobivari.flops(resolutions)
#         flops += self.norm1.flops(resolutions)
#         flops += self.ffn.flops(resolutions)
#         flops += self.norm2.flops(resolutions)
#         params = sum([p.numel() for n,p in self.named_parameters()])
#         return flops

# class EncoderStage(nn.Module):
#     def __init__(self, depth, dim, num_head, chsa_head_ratio, FFAT, window_size=8, head_dim=None,
#                  qkv_bias=True, mv_ver=1, hidden_ratio=2.0, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm,
#                  attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True, mv_act=nn.LeakyReLU,):
#         super(EncoderStage, self).__init__()

#         self.depth = depth
#         self.dim = dim
#         self.num_head = num_head
#         self.window_size = window_size
#         shift = window_size//2

#         self.blocks = nn.ModuleList()
#         for d in range(depth):
#             self.blocks.add_module(f'block{d}', DRAMiTransformer(dim, num_head, chsa_head_ratio, FFAT,
#                                                                  window_size, 0 if d%2==0 else shift, head_dim, qkv_bias,
#                                                                  mv_ver, hidden_ratio, act_layer, norm_layer,
#                                                                  attn_drop, proj_drop, drop_path, helper, mv_act,
#                                                                  ))
#     def forward(self, x):
#         sp, ch = None, None
#         for i, blk in enumerate(self.blocks):
#             x, sp, ch, attn = blk(x, sp, ch)
#             sp, ch = (None, None) if (sp.size(1)==0 or ch.size(1)==0) else (sp, ch) # pure SPSA or CHSA
#         return x, attn

#     def flops(self, resolutions):
#         flops = 0
#         for blk in self.blocks:
#             flops += blk.flops(resolutions)
#         return flops

# class Downsizing(nn.Module):
#     """ Patch Merging Layer.

#     Args:
#         dim (int): Number of input dimension (channels).
#         downsample_dim (int, optional): Number of output dimension (channels) (dim if not set).  Default: None
#         norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
#     """

#     def __init__(self, dim, downsample_dim=None, norm_layer=ReshapeLayerNorm, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
#         super(Downsizing, self).__init__()
#         self.dim = dim
#         self.downsample_dim = downsample_dim or dim
#         self.norm = norm_layer(4*dim)
#         if mv_ver==3:
#             self.reduction = MobiVari1(4*dim, 3, 1, act=mv_act, out_dim=self.downsample_dim)
#         elif mv_ver==2:
#             self.reduction = MobiVari2(4*dim, 3, 1, act=mv_act, out_dim=self.downsample_dim, exp_factor=exp_factor, expand_groups=expand_groups)

#             # NAFBlock in not accepted
#             # NAFBlock---------------------------------------------
#         elif mv_ver==1:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------

#     def forward(self, x):
#         B, C, H, W = x.size()

#         # Concat 2x2
#         x0 = x[:, :, 0::2, 0::2]  # [B, C, H/2, W/2], top-left
#         x1 = x[:, :, 0::2, 1::2]  # [B, C, H/2, W/2], top-right
#         x2 = x[:, :, 1::2, 0::2]  # [B, C, H/2, W/2], bottom-left
#         x3 = x[:, :, 1::2, 1::2]  # [B, C, H/2, W/2], bottom-right
#         x = torch.cat([x0, x1, x2, x3], dim=1)  # [B, 4C, H/2, W/2]
#         return self.reduction(self.norm(x)) # [B, C, H/2, W/2]

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = self.norm.flops((H//2,W//2)) + self.reduction.flops((H//2,W//2))
#         return flops

# class Bottleneck(nn.Module):
#     def __init__(self, dim, num_stages, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
#         super(Bottleneck, self).__init__()

#         self.dim = dim
#         self.cat_dim = sum([dim//(4**x) for x in range(num_stages)])
#         if mv_ver==1:
#             self.mobivari = MobiVari1(self.cat_dim, 3, 1, act=mv_act, out_dim=dim)
#         elif mv_ver==2:
#             self.mobivari = MobiVari2(self.cat_dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)

#             # NAFBlock---------------------------------------------
#         elif mv_ver==3:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------
#         self.act = act_layer()
#         self.norm = norm_layer(dim)

#     def shallow_down(self, x, r):
#         for _ in range(r):
#             x = F.max_pool2d(x, 2)
#         return F.leaky_relu(x)

#     def forward(self, x_list):
#         xs = x_list[0]
#         new_x = []
#         for i in range(len(x_list[1:])):
#             x_ = x_list[i+1]+self.shallow_down(xs, i)
#             new_x.append(F.pixel_shuffle(x_, 2**i))
#         new_x = self.norm(self.mobivari(torch.cat(new_x, dim=1)))
#         return new_x

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = 0
#         # self.shallow_down (iterative max-pool)
#         flops += (H//2)*(W//2) * 2*2 * self.dim # shallow-down into stage2 output (iter1)
#         flops += (H//2)*(W//2) * 2*2 * self.dim # shallow-down into stage3 output (iter1)
#         flops += (H//4)*(W//4) * 2*2 * self.dim # shallow-down into stage3 output (iter2)
#         flops += self.mobivari.flops((H,W))
#         flops += self.norm.flops((H,W))
#         return flops

# class HRAMi(nn.Module):
#     def __init__(self, dim, kernel_size=3, stride=1, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
#         super(HRAMi, self).__init__()

#         self.dim = dim
#         self.kernel_size = kernel_size

#         if mv_ver==1:
#             self.mobivari = MobiVari1(dim+dim//4+dim//16+dim, kernel_size, stride, act=mv_act, out_dim=dim)
#         elif mv_ver==2:
#             self.mobivari = MobiVari2(dim+dim//4+dim//16+dim, kernel_size, stride, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
#             # NAFBlock---------------------------------------------
#         elif mv_ver==3:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------

#     def forward(self, attn_list):
#         for i, attn in enumerate(attn_list[:-1]):
#             attn = F.pixel_shuffle(attn, 2**i)
#             x = attn if i==0 else torch.cat([x, attn], dim=1)
#         x = torch.cat([x, attn_list[-1]], dim=1)
#         x = self.mobivari(x)
#         return x

#     def flops(self, resolutions):
#         return self.mobivari.flops(resolutions)

# class Reconstruction(nn.Module):
#     def __init__(self, target_mode, dim, kernel_size=3, stride=1, num_mv=2, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
#         super(Reconstruction, self).__init__()

#         self.upscale = upscale = int(target_mode[-1]) if target_mode in ['light_x2', 'light_x3', 'light_x4'] else 1
#         self.target_mode = target_mode
#         self.dim = dim
#         self.out_chans = 3 if 'gray' not in target_mode else 1
#         self.kernel_size = kernel_size
#         self.stride = stride
#         self.num_mv = num_mv

#         self.mobivari = nn.ModuleList()
#         for i in range(num_mv):
#             if mv_ver==3:
#                 self.mobivari.add_module(f'mobivari{i}', MobiVari1(dim, kernel_size, stride, mv_act))
#             elif mv_ver==2:
#                 self.mobivari.add_module(f'mobivari{i}', MobiVari2(dim, kernel_size, stride, mv_act, None, exp_factor, expand_groups))
#                 # NAFBlock---------------------------------------------
#             elif mv_ver==1:
#                  self.mobivari.add_module(f'mobivari{i}',NAFBlock(c=self.dim))
#                 # -------------------------------------------------------


#         self.conv = nn.Conv2d(dim, self.out_chans*(upscale**2), kernel_size, stride, kernel_size//2)
#         self.shuffle = nn.PixelShuffle(upscale) if upscale!=1 else nn.Identity()
#         self.final_conv = nn.Conv2d(self.out_chans, self.out_chans, kernel_size, stride, kernel_size//2)

#     def forward(self, x):
#         for mobivari in self.mobivari:
#             x = mobivari(x)
#         return self.final_conv(self.shuffle(self.conv(x)))

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = 0
#         for mobivari in self.mobivari: # self.mobivari
#             flops += mobivari.flops((H,W))
#         flops += H*W * self.kernel_size*self.kernel_size * self.dim * self.out_chans*(self.upscale**2) # self.conv
#         flops += H*W * self.kernel_size*self.kernel_size * self.out_chans * self.out_chans # self.final_conv
#         return flops

# class RAMiT_FFAT(nn.Module):
#     def __init__(self, in_chans=3, dim=64, depths=(1,4,4,6), num_heads=(4,4,4,4), head_dim=None, chsa_head_ratio=0.25, #depths = (6,4,4,6)
#                  window_size=8, hidden_ratio=2.0, qkv_bias=True, mv_ver=1, exp_factor=1.2, expand_groups=4,
#                  act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, tail_mv=2, weight_init='', target_mode='light_x2', img_norm=True,
#                  attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True, mv_act=nn.LeakyReLU,
#                  num_blocks = [1,4,6,6,8],
#                  heads = [1,2,4,8],
#                  ffn_expansion_factor = 2.66,
#                  bias = False,
#                  LayerNorm_type = 'WithBias',   ## Other option 'BiasFree'
#                  FFAT = False, **kwargs):
#         super(RAMiT_FFAT, self).__init__()
#         self.cat_dim = sum([dim//(4**x) for x in range(len(depths)-1)]) # added to use the torchinfor.summary
#         self.unit = 2**(len(depths)-2)*window_size

#         self.in_chans = in_chans
#         self.dim = dim
#         self.depths = depths
#         self.num_heads = num_heads
#         self.head_dim = head_dim
#         self.window_size = window_size
#         self.hidden_ratio = hidden_ratio
#         self.qkv_bias = qkv_bias
#         self.act_layer = act_layer
#         norm_layer = ReshapeLayerNorm if norm_layer == 'ReshapeLayerNorm' else norm_layer
#         self.norm_layer = norm_layer = ReshapeLayerNorm if norm_layer == 'ReshapeLayerNorm' else norm_layer
#         self.tail_mv = tail_mv

#         self.scale = int(target_mode[-1]) if target_mode in ['light_x2', 'light_x3', 'light_x4'] else 1
#         self.mean, self.std = mean_std(self.scale, target_mode)
#         self.target_mode = target_mode
#         self.img_norm = img_norm

#         dpr = [x.item() for x in torch.linspace(0, drop_path, sum(depths))]  # stochastic depth decay rule

#         self.shallow = ShallowModule(in_chans, dim, 3, 1)
#         self.stage1 = EncoderStage(depths[0], dim, num_heads[0], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
#                                    hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)


#         # FFAT
#         self.encoder_level1 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])

#         self.encoder_level2 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])

#         self.encoder_level3 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])
#         # FFAT


#         self.down1 = Downsizing(dim, dim, norm_layer, mv_ver, mv_act)
#         self.stage2 = EncoderStage(depths[0], dim, num_heads[1], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
#                                    hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)
#         self.down2 = Downsizing(dim, dim, norm_layer, mv_ver, mv_act)
#         self.stage3 = EncoderStage(depths[0], dim, num_heads[2], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
#                                    hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)





#         if mv_ver==3:
#             self.mobivari = MobiVari1(self.cat_dim, 3, 1, act=mv_act, out_dim=dim)
#         elif mv_ver==2:
#             self.mobivari = MobiVari2(self.cat_dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
#             # NAFBlock not Accepted
#             # NAFBlock---------------------------------------------
#         elif mv_ver==1:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------
#         self.act = act_layer()
#         self.norm = norm_layer(dim)
#         # self.bottleneck = Bottleneck(dim, len(depths)-1, act_layer, norm_layer, mv_ver, mv_act)

#         self.stage4 = EncoderStage(depths[0], dim, num_heads[3], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
#                                    hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)







#         if mv_ver==3:
#             self.mobivari_mix = MobiVari1(dim+dim//4+dim//16+dim, 3, 1, act=mv_act, out_dim=dim)
#         elif mv_ver==2:
#             self.mobivari_mix = MobiVari2(dim+dim//4+dim//16+dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
#             # NAFBlock not accepted
#              # NAFBlock---------------------------------------------
#         elif mv_ver==1:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------
#         # self.attn_mix = HRAMi(dim, 3, 1, mv_ver, mv_act)


#         self.to_target = Reconstruction(target_mode, dim, 3, 1, tail_mv, mv_ver, mv_act, exp_factor, expand_groups)

#         self.apply(self._init_weights)

#     def forward_size_norm(self, x):
#         _,_,h,w = x.size()
#         padh = self.unit-(h%self.unit) if h%self.unit!=0 else 0
#         padw = self.unit-(w%self.unit) if w%self.unit!=0 else 0
#         x = TF.pad(x, (0,0,padw,padh))
#         return TF.normalize(x, self.mean, self.std) if self.img_norm else x

#     def forward(self, x):
#         _,_,H_ori,W_ori = x.size()
#         x = self.forward_size_norm(x)
#         B, C, H, W = x.size()
#         x_lq = x if self.scale == 1 else None # due to residual image (except for only SR)

#         shallow = self.shallow(x)
#         o1_, attn1 = self.stage1(shallow) # [B, C, H, W]


#         # FFAT
#         o1_ = self.encoder_level1(o1_)
#         # FFAT


#         o1 = self.down1(o1_) # [B, C, H//2, W//2]

#         o2_, attn2 = self.stage2(o1) # [B, C, H//2, W//2]


#         # FFAT
#         o2_ = self.encoder_level2(o2_)
#         # FFAT


#         o2 = self.down2(o2_) # [B, C, H//4, W//4]

#         o3_, attn3 = self.stage3(o2) # [B, C, H//4, W//4]


#         # FFAT
#         o3_ = self.encoder_level3(o3_)
#         # FFAT




# # Bottleneck
#         def shallow_down( x, r):
#           for _ in range(r):
#               x = F.max_pool2d(x, 2)
#           return F.leaky_relu(x)
#         x_list = [shallow, o1_, o2_, o3_]
#         xs = x_list[0]
#         new_x = []
#         for i in range(len(x_list[1:])):
#             x_ = x_list[i+1]+shallow_down(xs, i)
#             new_x.append(F.pixel_shuffle(x_, 2**i))
#         ob = self.norm(self.mobivari(torch.cat(new_x, dim=1)))
#         # ob = self.bottleneck([shallow, o1_, o2_, o3_]) # [B, C, H, W]



#         o4, attn4 = self.stage4(ob) # [B, C, H, W]






# # HRAMiT
#         attn_list = [attn1, attn2, attn3, attn4]
#         for i, attn in enumerate(attn_list[:-1]):
#             attn = F.pixel_shuffle(attn, 2**i)
#             x = attn if i==0 else torch.cat([x, attn], dim=1)
#         x = torch.cat([x, attn_list[-1]], dim=1)
#         mix = self.mobivari_mix(x)
#         # mix = self.attn_mix([attn1, attn2, attn3, attn4]) # [B, C, H, W]



#         o4 = o4 * mix # [B, C, H, W]

#         out = self.to_target(o4+shallow) # global skip connection
#         out = out + x_lq if self.scale == 1 else out # residual output (except for only SR)

#         out = denormalize(out, self.mean, self.std) if self.img_norm else out
#         out = out[:, :, :H_ori*self.scale, :W_ori*self.scale]
#         return out

#     def flops(self, resolutions):
#         H_ori,W_ori = resolutions
#         padh = self.unit-(H_ori%self.unit) if H_ori%self.unit!=0 else 0
#         padw = self.unit-(W_ori%self.unit) if W_ori%self.unit!=0 else 0
#         H,W = H_ori+padh,W_ori+padw
#         flops = 0
#         flops += self.shallow.flops((H,W))
#         flops += self.stage1.flops((H,W))
#         flops += self.down1.flops((H,W))
#         flops += self.stage2.flops((H//2,W//2))
#         flops += self.down2.flops((H//2,W//2))
#         flops += self.stage3.flops((H//4,W//4))
#         flops += self.bottleneck.flops((H,W))
#         flops += self.stage4.flops((H,W))
#         flops += self.attn_mix.flops((H,W))
#         flops += self.dim * H*W # o4 = o4*mix
#         flops += self.to_target.flops((H,W))
#         return flops

#     def _init_weights(self, m):
#         # Swin V2 manner
#         if isinstance(m, nn.Linear):
#             trunc_normal_(m.weight, std=.02)
#             if isinstance(m, nn.Linear) and m.bias is not None:
#                 nn.init.constant_(m.bias, 0)
#         elif isinstance(m, nn.LayerNorm):
#             nn.init.constant_(m.bias, 0)
#             nn.init.constant_(m.weight, 1.0)

#     @torch.jit.ignore
#     def no_weight_decay(self):
#         nwd = set()
#         for n, _ in self.named_parameters():
#             if 'relative_position_bias_table' in n:
#                 nwd.add(n)
#         return nwd






# # ----------------------------Shadow_R-------------------------
#     # @torch.no_grad()
#     # def _init_weights(module_list, scale=1, bias_fill=0, **kwargs):
#     #   """Initialize network weights.

#     #   Args:
#     #       module_list (list[nn.Module] | nn.Module): Modules to be initialized.
#     #       scale (float): Scale initialized weights, especially for residual
#     #           blocks. Default: 1.
#     #       bias_fill (float): The value to fill bias. Default: 0
#     #       kwargs (dict): Other arguments for initialization function.
#     #   """
#     #   if not isinstance(module_list, list):
#     #       module_list = [module_list]
#     #   for module in module_list:
#     #       for m in module.modules():
#     #           if isinstance(m, nn.Conv2d):
#     #               nn.init.kaiming_normal_(m.weight, **kwargs)
#     #               m.weight.data *= scale
#     #               if m.bias is not None:
#     #                   m.bias.data.fill_(bias_fill)
#     #           elif isinstance(m, nn.Linear):
#     #               nn.init.kaiming_normal_(m.weight, **kwargs)
#     #               m.weight.data *= scale
#     #               if m.bias is not None:
#     #                   m.bias.data.fill_(bias_fill)
#     #           elif isinstance(m, _BatchNorm):
#     #               nn.init.constant_(m.weight, 1)
#     #               if m.bias is not None:
#     #                   m.bias.data.fill_(bias_fill)

In [6]:
# RAMiT (Reciprocal Attention Mixing Transformer)

from pdb import set_trace as stx
import numbers
from skimage import io
from skimage.metrics import peak_signal_noise_ratio as psnr





from torch import nn, Tensor
from einops import rearrange
import torch.nn.functional as F
import torch
from torchvision.transforms import functional as TF
import math
from RAMiT.my_model.common.mean_std import mean_std
from RAMiT.utils.etc_utils import denormalize
from timm.models.fx_features import register_notrace_function
from timm.models.layers import trunc_normal_, to_2tuple
from torch.nn.modules.batchnorm import _BatchNorm
def make_model(args, opts, pe):
    model = RAMiT(target_mode=args.target_mode,
                  img_norm=args.img_norm,
                  in_chans=opts['in_chans'],
                  dim = opts['dim'],
                  depths = opts['depths'],
                  num_heads = opts['num_heads'],
                  head_dim = opts['head_dim'],
                  chsa_head_ratio = opts['chsa_head_ratio'],
                  window_size = opts['window_size'],
                  hidden_ratio = opts['hidden_ratio'],
                  qkv_bias = opts['qkv_bias'],
                  mv_ver = opts['mv_ver'],
                  exp_factor=opts['exp_factor'],
                  expand_groups=opts['expand_groups'],
                  act_layer = opts['act_layer'],
                  norm_layer = opts['norm_layer'],
                  tail_mv = opts['tail_mv'],
                  attn_drop = opts['attn_drop'],
                  proj_drop = opts['proj_drop'],
                  drop_path = opts['drop_path'],
                  helper = opts['helper'],
                  mv_act = opts['mv_act']
                 )
    num_params = sum([p.numel() for n,p in model.named_parameters()])
    print(format(num_params, ','))
    return model






# FFSANet------------------------------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft

class BasicConv(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, bias=True, norm=False, relu=True, transpose=False):
        super(BasicConv, self).__init__()
        if bias and norm:
            bias = False

        padding = kernel_size // 2
        layers = list()
        if transpose:
            padding = kernel_size // 2 -1
            layers.append(nn.ConvTranspose2d(in_channel, out_channel, kernel_size, padding=padding, stride=stride, bias=bias))
        else:
            layers.append(
                nn.Conv2d(in_channel, out_channel, kernel_size, padding=padding, stride=stride, bias=bias))
        if norm:
            layers.append(nn.BatchNorm2d(out_channel))
        if relu:
            layers.append(nn.GELU())
        self.main = nn.Sequential(*layers)

    def forward(self, x):
        return self.main(x)

class ResBlock(nn.Module):
    def __init__(self, in_channel, out_channel, filter=False):
        super(ResBlock, self).__init__()
        self.conv1 = BasicConv(in_channel, out_channel, kernel_size=3, stride=1, relu=True)
        self.conv2 = BasicConv(out_channel, out_channel, kernel_size=3, stride=1, relu=False)
        if filter:
            self.cubic_11 = cubic_attention(in_channel//2, group=1, kernel=11)
            self.cubic_7 = cubic_attention(in_channel//2, group=1, kernel=7)
        self.filter = filter

    def forward(self, x):
        out = self.conv1(x)
        if self.filter:
            out = torch.chunk(out, 2, dim=1)
            out_11 = self.cubic_11(out[0])
            out_7 = self.cubic_7(out[1])
            out = torch.cat((out_11, out_7), dim=1)
        out = self.conv2(out)
        return out + x

# class cubic_attention(nn.Module):
#     def __init__(self, dim, group, kernel) -> None:
#         super().__init__()

#         self.H_spatial_att = spatial_strip_att(dim, group=group, kernel=kernel)
#         self.W_spatial_att = spatial_strip_att(dim, group=group, kernel=kernel, H=False)
#         self.gamma = nn.Parameter(torch.zeros(dim,1,1))
#         self.beta = nn.Parameter(torch.ones(dim,1,1))

#     def forward(self, x):
#         # Apply Fourier Transform to the input
#         x_fft = torch.fft.fft2(x)

#         # Apply attention mechanisms in the frequency domain
#         out = self.H_spatial_att(x_fft)
#         out = self.W_spatial_att(out)

#         # Apply Inverse Fourier Transform to get back to the spatial domain
#         out_ifft = torch.fft.ifft2(out).real

#         return self.gamma * out_ifft + x * self.beta

class cubic_attention(nn.Module):
    def __init__(self, dim, group, kernel) -> None:
        super().__init__()

        self.H_spatial_att = spatial_strip_att(dim, group=group, kernel=kernel)
        self.W_spatial_att = spatial_strip_att(dim, group=group, kernel=kernel, H=False)
        self.gamma = nn.Parameter(torch.zeros(dim,1,1))
        self.beta = nn.Parameter(torch.ones(dim,1,1))

    def forward(self, x):
        # Apply Fourier Transform to the input
        x_fft = torch.fft.fft2(x)

        # Separate real and imaginary parts
        real_part = x_fft.real
        imag_part = x_fft.imag

        # Apply attention mechanisms to real and imaginary parts separately
        out_real = self.H_spatial_att(real_part)
        out_real = self.W_spatial_att(out_real)
        out_imag = self.H_spatial_att(imag_part)
        out_imag = self.W_spatial_att(out_imag)

        # Combine real and imaginary parts back into a complex tensor
        out = torch.complex(out_real, out_imag)

        # Apply Inverse Fourier Transform to get back to the spatial domain
        out_ifft = torch.fft.ifft2(out).real

        return self.gamma * out_ifft + x * self.beta


class spatial_strip_att(nn.Module):
    def __init__(self, dim, kernel=5, group=2, H=True) -> None:
        super().__init__()

        self.k = kernel
        pad = kernel // 2
        self.kernel = (1, kernel) if H else (kernel, 1)
        self.padding = (kernel//2, 1) if H else (1, kernel//2)

        self.group = group
        self.pad = nn.ReflectionPad2d((pad, pad, 0, 0)) if H else nn.ReflectionPad2d((0, 0, pad, pad))
        self.conv = nn.Conv2d(dim, group*kernel, kernel_size=1, stride=1, bias=False)
        self.ap = nn.AdaptiveAvgPool2d((1, 1))
        self.filter_act = nn.Sigmoid()

    def forward(self, x):
        filter = self.ap(x)
        filter = self.conv(filter)
        n, c, h, w = x.shape

        # Perform operations in the frequency domain
        x = F.unfold(self.pad(x), kernel_size=self.kernel).reshape(n, self.group, c//self.group, self.k, h*w)
        n, c1, p, q = filter.shape
        filter = filter.reshape(n, c1//self.k, self.k, p*q).unsqueeze(2)
        filter = self.filter_act(filter)
        out = torch.sum(x * filter, dim=3).reshape(n, c, h, w)

        return out


# FFSANet----------------------------------------------------------

class ShallowModule(nn.Module):
    def __init__(self, in_chans, out_chans, kernel_size=3, stride=1):
        super(ShallowModule, self).__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.kernel_size = kernel_size
        self.stride = stride

        self.conv = nn.Conv2d(in_chans, out_chans, kernel_size, stride, kernel_size//2)

    def forward(self, x):
        return self.conv(x)

    def flops(self, resolutions):
        return resolutions[0]*resolutions[1] * self.kernel_size*self.kernel_size * self.in_chans * self.out_chans

class QKVProjection(nn.Module):
    def __init__(self, dim, num_head, qkv_bias=True):
        super(QKVProjection, self).__init__()
        self.dim = dim
        self.num_head = num_head

        self.qkv = nn.Conv2d(dim, 3*dim, 1, bias=qkv_bias)

    def forward(self, x):
        B, C, H, W = x.size()
        qkv = self.qkv(x)
        qkv = rearrange(qkv, 'b (l c) h w -> b l c h w', l=self.num_head)
        return qkv

    def flops(self, resolutions):
        return resolutions[0]*resolutions[1] * 1*1 * self.dim * 3*self.dim

def get_relative_position_index(win_h, win_w):
    # get pair-wise relative position index for each token inside the window
    coords = torch.stack(torch.meshgrid([torch.arange(win_h), torch.arange(win_w)]))  # 2, Wh, Ww
    coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
    relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww (xaxis matrix & yaxis matrix)
    relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
    relative_coords[:, :, 0] += win_h - 1  # shift to start from 0
    relative_coords[:, :, 1] += win_w - 1
    relative_coords[:, :, 0] *= 2 * win_w - 1
    return relative_coords.sum(-1)  # Wh*Ww, Wh*Ww

class SpatialSelfAttention(nn.Module):
    def __init__(self, dim, num_head, total_head, window_size=8, shift=0, attn_drop=0.0, proj_drop=0.0, helper=True):
        super(SpatialSelfAttention, self).__init__()
        self.dim = dim
        self.num_head = num_head
        self.total_head = total_head
        self.window_size = window_size
        self.window_area = window_size**2
        self.shift = shift
        self.helper = helper

        self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_head, 1, 1))), requires_grad=True)

        # define a parameter table of relative position bias, shape: 2*Wh-1 * 2*Ww-1, nH
        self.relative_position_bias_table = nn.Parameter(torch.zeros((2 * window_size - 1) * (2 * window_size - 1), num_head))

        # get pair-wise relative position index for each token inside the window
        self.register_buffer("relative_position_index", get_relative_position_index(window_size, window_size))

        self.attn_drop = nn.Dropout(attn_drop)

        self.proj = nn.Conv2d(dim*num_head, dim*num_head, 1)
        self.proj_drop = nn.Dropout(proj_drop)

    def _get_rel_pos_bias(self) -> torch.Tensor:
        relative_position_bias = self.relative_position_bias_table[
            self.relative_position_index.view(-1)].view(self.window_area, self.window_area, -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        return relative_position_bias.unsqueeze(0)

    def forward(self, qkv, ch=None):
        B, L, C, H, W = qkv.size()
        # window shift
        if self.shift > 0:
            qkv = torch.roll(qkv, shifts=(-self.shift, -self.shift), dims=(-2,-1))

        # window partition
        q,k,v = rearrange(qkv, 'b l c (h wh) (w ww) -> (b h w) l (wh ww) c',
                          wh=self.window_size, ww=self.window_size).chunk(3, dim=-1) # [B_, L1, hw, C/L] respectively
        if ch is not None and self.helper: # [B, C, H, W]
            if self.shift > 0:
                ch = torch.roll(ch, shifts=(-self.shift, -self.shift), dims=(-2,-1))
            ch = rearrange(ch, 'b (l c) (h wh) (w ww) -> (b h w) l (wh ww) c',
                           l=self.total_head-self.num_head, wh=self.window_size, ww=self.window_size) # [B_, L1, hw, C/L]
            ch = torch.mean(ch, dim=1, keepdim=True) # head squeeze [B_, 1, hw, C/L]
            v = v*ch # [B_, L1, hw, C/L]

        attn = F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(2,-1) # [B_, L1, hw, hw]
        logit_scale = torch.clamp(self.logit_scale, max=math.log(1. / 0.01)).exp()
        attn = attn * logit_scale

        attn = attn + self._get_rel_pos_bias()
        attn = self.attn_drop(F.softmax(attn, dim=-1))

        x = attn @ v # [B_, L1, hw, C/L]

        # window unpartition + head merge
        x = window_unpartition(x, (H,W), self.window_size) # [B, L1*C/L, H, W]
        x = self.proj_drop(self.proj(x))

        # window reverse shift
        if self.shift > 0:
            x = torch.roll(x, shifts=(self.shift, self.shift), dims=(-2,-1))

        return x

    def flops(self, resolutions):
        H,W = resolutions
        num_wins = H//self.window_size * W//self.window_size
        flops = self.num_head * H*W * self.dim if self.helper else 0 # v = v*ch
        flops += num_wins * self.num_head * self.window_area * self.dim * self.window_area # attn = Q@K^T
        flops += num_wins * self.num_head * self.window_area * self.window_area * self.dim # attn@V
        flops += H*W * 1*1 * self.num_head*self.dim * self.num_head*self.dim # self.proj
        return flops

@register_notrace_function
def window_unpartition(x, resolutions, window_size):
    return rearrange(x, '(b h w) l (wh ww) c -> b (l c) (h wh) (w ww)',
                     h=resolutions[0]//window_size, w=resolutions[1]//window_size, wh=window_size)

class ChannelSelfAttention(nn.Module):
    def __init__(self, dim, num_head, total_head, attn_drop=0.0, proj_drop=0.0, helper=True):
        super(ChannelSelfAttention, self).__init__()
        self.dim = dim
        self.num_head = num_head
        self.total_head = total_head
        self.helper = helper

        self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_head, 1, 1))), requires_grad=True)

        self.attn_drop = nn.Dropout(attn_drop)

        self.proj = nn.Conv2d(dim*num_head, dim*num_head, 1)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, qkv, sp=None):
        B, L, C, H, W = qkv.size()

        q,k,v = rearrange(qkv, 'b l c h w -> b l c (h w)').chunk(3, dim=-2) # [B, L2, C/L, HW]
        if sp is not None and self.helper:
            sp = torch.mean(sp, dim=1, keepdim=True) # channel squeeze # [B, 1, H, W]
            sp = rearrange(sp, 'b (l c) h w -> b l c (h w)', l=1) # [B, 1, 1, HW]
            v = v*sp # [B, L2, C/L, HW]

        attn = F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(2,-1) # [B, L2, C/L, C/L]
        logit_scale = torch.clamp(self.logit_scale, max=math.log(1. / 0.01)).exp()
        attn = attn * logit_scale

        attn = F.softmax(attn, dim=-1)
        attn = self.attn_drop(attn)

        x = attn @ v # [B, L2, C/L, HW]

        # head merge
        x = rearrange(x, 'b l c (h w) -> b (l c) h w', h=H) # [B, L2*C/L, H, W]
        x = self.proj_drop(self.proj(x)) # [B, L2*C/L, H, W]

        return x

    def flops(self, resolutions):
        H,W = resolutions
        flops = self.num_head * self.dim * H*W if self.helper else 0 # v = v*sp
        flops += self.num_head * self.dim * H*W * self.dim # attn = Q@K^T
        flops += self.num_head * self.dim * self.dim * H*W # attn@V
        flops += H*W * 1*1 * self.num_head*self.dim * self.num_head*self.dim # self.proj
        return flops

class ReshapeLayerNorm(nn.Module):
    def __init__(self, dim, norm_layer=nn.LayerNorm):
        super(ReshapeLayerNorm, self).__init__()

        self.dim = dim
        self.norm = norm_layer(dim)

    def forward(self, x):
        B, C, H, W = x.size()
        x = rearrange(x, 'b c h w -> b (h w) c')
        x = self.norm(x)
        x = rearrange(x, 'b (h w) c -> b c h w', h=H)
        return x

    def flops(self, resolutions):
        H,W = resolutions
        flops = 0
        flops += H*W * self.dim
        return flops

class MobiVari1(nn.Module): # MobileNet v1 Variants
    def __init__(self, dim, kernel_size, stride, act=nn.LeakyReLU, out_dim=None):
        super(MobiVari1, self).__init__()
        self.dim = dim
        self.kernel_size = kernel_size
        self.out_dim = out_dim or dim

        self.dw_conv = nn.Conv2d(dim, dim, kernel_size, stride, kernel_size//2, groups=dim)
        self.pw_conv = nn.Conv2d(dim, self.out_dim, 1, 1, 0)
        self.act = act()

    def forward(self, x):
        out = self.act(self.pw_conv(self.act(self.dw_conv(x))+x))
        return out + x if self.dim==self.out_dim else out

    def flops(self, resolutions):
        H,W = resolutions
        flops = H*W * self.kernel_size*self.kernel_size * self.dim  +  H*W * 1*1 * self.dim * self.out_dim # self.dw_conv + self.pw_conv
        return flops

class MobiVari2(MobiVari1): # MobileNet v2 Variants
    def __init__(self, dim, kernel_size, stride, act=nn.LeakyReLU, out_dim=None, exp_factor=1.2, expand_groups=4):
        super(MobiVari2, self).__init__(dim, kernel_size, stride, act, out_dim)
        self.expand_groups = expand_groups
        expand_dim = int(dim*exp_factor)
        expand_dim = expand_dim+(expand_groups-expand_dim%expand_groups)
        self.expand_dim = expand_dim

        self.exp_conv = nn.Conv2d(dim, self.expand_dim, 1, 1, 0, groups=expand_groups)
        self.dw_conv = nn.Conv2d(expand_dim, expand_dim, kernel_size, stride, kernel_size//2, groups=expand_dim)
        self.pw_conv = nn.Conv2d(expand_dim, self.out_dim, 1, 1, 0)

    def forward(self, x):
        x1 = self.act(self.exp_conv(x))
        out = self.pw_conv(self.act(self.dw_conv(x1)+x1))
        return out + x if self.dim==self.out_dim else out

    def flops(self, resolutions):
        H,W = resolutions
        flops = H*W * 1*1 * (self.dim//self.expand_groups) * self.expand_dim # self.exp_conv
        flops += H*W * self.kernel_size*self.kernel_size * self.expand_dim # self.dw_conv
        flops += H*W * 1*1 * self.expand_dim * self.out_dim # self.pw_conv
        return flops

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_ratio, act_layer=nn.GELU, bias=True, drop=0.0):
        super(FeedForward, self).__init__()

        self.dim = dim
        self.hidden_ratio = hidden_ratio

        self.hidden = nn.Conv2d(dim, int(dim*hidden_ratio), 1, bias=bias)
        self.drop1 = nn.Dropout(drop)
        self.out = nn.Conv2d(int(dim*hidden_ratio), dim, 1, bias=bias)
        self.drop2 = nn.Dropout(drop)
        self.act = act_layer()

    def forward(self, x):
        return self.drop2(self.out(self.drop1(self.act(self.hidden(x)))))

    def flops(self, resolutions):
        H,W = resolutions
        flops = 2 * H*W * 1*1 * self.dim * self.dim*self.hidden_ratio # self.hidden + self.out
        return flops

class NoLayer(nn.Identity):
    def __init__(self):
        super(NoLayer, self).__init__()
    def flops(self, resolutions):
        return 0
    def forward(self, x, **kwargs):
        return x.flatten(1,2)

class DRAMiTransformer(nn.Module): # Reciprocal Attention Transformer Block
    def __init__(self, dim, num_head, chsa_head_ratio, FFAT, window_size=8, shift=0, head_dim=None, qkv_bias=True, mv_ver=1,
                 hidden_ratio=2.0, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True,
                 mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4,
                 num_blocks = [4,6,6,8], heads = [1,2,4,8], ffn_expansion_factor = 2.66, bias = False, LayerNorm_type = 'WithBias'):
        super(DRAMiTransformer, self).__init__()

        self.dim = dim
        self.num_head = num_head
        self.window_size = window_size
        self.chsa_head = int(num_head*chsa_head_ratio)
        self.shift = shift
        self.helper = helper
        self.FFAT = FFAT

        self.qkv_proj = QKVProjection(dim, num_head, qkv_bias=qkv_bias)
        self.sp_attn = SpatialSelfAttention(dim//num_head, num_head-self.chsa_head, num_head,
                                            window_size, shift, attn_drop, proj_drop, helper) if num_head-self.chsa_head != 0 else NoLayer()
        self.ch_attn = ChannelSelfAttention(dim//num_head, self.chsa_head, num_head, attn_drop, proj_drop, helper) if self.chsa_head != 0 else NoLayer()


        #FFAT
        if self.FFAT:
          self.encoder_level1 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])
        #FFAT


        if mv_ver==3:
            self.mobivari = MobiVari1(dim, 3, 1, act=mv_act)
        elif mv_ver==2:
            self.mobivari = MobiVari2(dim, 3, 1, act=mv_act, out_dim=None, exp_factor=exp_factor, expand_groups=expand_groups)
            # NAFBlock---------------------------------------------
        elif mv_ver==1:
            self.mobivari = NAFBlock(c=self.dim)
            # -------------------------------------------------------

        self.norm1 = norm_layer(dim)

        self.ffn = FeedForward(dim, hidden_ratio, act_layer=act_layer)
        self.norm2 = norm_layer(dim)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x, sp_=None, ch_=None):
        B, C, H, W = x.size()

        # QKV projection + head split
        qkv = self.qkv_proj(x) # [B, L, C, H, W]

        # SP-SA / CH-SA
        sp = self.sp_attn(qkv[:,:self.num_head-self.chsa_head], ch=ch_) # [B, L1*C/L, H, W]
        ch = self.ch_attn(qkv[:,self.num_head-self.chsa_head:], sp=sp_) # [B, L2*C/L, H, W]


        #FFAT
        if self.FFAT:
          FFAT_block = self.encoder_level1(torch.cat([sp, ch], dim=1))
          attn0 = self.mobivari(FFAT_block)
        #FFAT


        else:
          attn0 = self.mobivari(torch.cat([sp, ch], dim=1)) # merge [B, C, H, W]
        attn = self.drop_path(self.norm1(attn0)) + x # LN, skip connection [B, C, H, W]

        # FFN
        out = self.drop_path(self.norm2(self.ffn(attn))) + attn # FFN, LN, skip connection [B, C, H, W]

        return out, sp, ch, attn0

    def flops(self, resolutions):
        flops = self.qkv_proj.flops(resolutions)
        flops += self.sp_attn.flops(resolutions)
        flops += self.ch_attn.flops(resolutions)
        flops += self.mobivari.flops(resolutions)
        flops += self.norm1.flops(resolutions)
        flops += self.ffn.flops(resolutions)
        flops += self.norm2.flops(resolutions)
        params = sum([p.numel() for n,p in self.named_parameters()])
        return flops

class EncoderStage(nn.Module):
    def __init__(self, depth, dim, num_head, chsa_head_ratio, FFAT, window_size=8, head_dim=None,
                 qkv_bias=True, mv_ver=1, hidden_ratio=2.0, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm,
                 attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True, mv_act=nn.LeakyReLU,):
        super(EncoderStage, self).__init__()

        self.depth = depth
        self.dim = dim
        self.num_head = num_head
        self.window_size = window_size
        shift = window_size//2

        self.blocks = nn.ModuleList()
        for d in range(depth):
            self.blocks.add_module(f'block{d}', DRAMiTransformer(dim, num_head, chsa_head_ratio, FFAT,
                                                                 window_size, 0 if d%2==0 else shift, head_dim, qkv_bias,
                                                                 mv_ver, hidden_ratio, act_layer, norm_layer,
                                                                 attn_drop, proj_drop, drop_path, helper, mv_act,
                                                                 ))
    def forward(self, x):
        sp, ch = None, None
        for i, blk in enumerate(self.blocks):
            x, sp, ch, attn = blk(x, sp, ch)
            sp, ch = (None, None) if (sp.size(1)==0 or ch.size(1)==0) else (sp, ch) # pure SPSA or CHSA
        return x, attn

    def flops(self, resolutions):
        flops = 0
        for blk in self.blocks:
            flops += blk.flops(resolutions)
        return flops

class Downsizing(nn.Module):
    """ Patch Merging Layer.

    Args:
        dim (int): Number of input dimension (channels).
        downsample_dim (int, optional): Number of output dimension (channels) (dim if not set).  Default: None
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, downsample_dim=None, norm_layer=ReshapeLayerNorm, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
        super(Downsizing, self).__init__()
        self.dim = dim
        self.downsample_dim = downsample_dim or dim
        self.norm = norm_layer(4*dim)
        if mv_ver==3:
            self.reduction = MobiVari1(4*dim, 3, 1, act=mv_act, out_dim=self.downsample_dim)
        elif mv_ver==2:
            self.reduction = MobiVari2(4*dim, 3, 1, act=mv_act, out_dim=self.downsample_dim, exp_factor=exp_factor, expand_groups=expand_groups)

            # NAFBlock in not accepted
            # NAFBlock---------------------------------------------
        elif mv_ver==1:
            self.mobivari = NAFBlock(c=self.dim)
            # -------------------------------------------------------

    def forward(self, x):
        B, C, H, W = x.size()

        # Concat 2x2
        x0 = x[:, :, 0::2, 0::2]  # [B, C, H/2, W/2], top-left
        x1 = x[:, :, 0::2, 1::2]  # [B, C, H/2, W/2], top-right
        x2 = x[:, :, 1::2, 0::2]  # [B, C, H/2, W/2], bottom-left
        x3 = x[:, :, 1::2, 1::2]  # [B, C, H/2, W/2], bottom-right
        x = torch.cat([x0, x1, x2, x3], dim=1)  # [B, 4C, H/2, W/2]
        return self.reduction(self.norm(x)) # [B, C, H/2, W/2]

    def flops(self, resolutions):
        H,W = resolutions
        flops = self.norm.flops((H//2,W//2)) + self.reduction.flops((H//2,W//2))
        return flops

class Bottleneck(nn.Module):
    def __init__(self, dim, num_stages, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
        super(Bottleneck, self).__init__()

        self.dim = dim
        self.cat_dim = sum([dim//(4**x) for x in range(num_stages)])
        if mv_ver==1:
            self.mobivari = MobiVari1(self.cat_dim, 3, 1, act=mv_act, out_dim=dim)
        elif mv_ver==2:
            self.mobivari = MobiVari2(self.cat_dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)

            # NAFBlock---------------------------------------------
        elif mv_ver==3:
            self.mobivari = NAFBlock(c=self.dim)
            # -------------------------------------------------------
        self.act = act_layer()
        self.norm = norm_layer(dim)

    def shallow_down(self, x, r):
        for _ in range(r):
            x = F.max_pool2d(x, 2)
        return F.leaky_relu(x)

    def forward(self, x_list):
        xs = x_list[0]
        new_x = []
        for i in range(len(x_list[1:])):
            x_ = x_list[i+1]+self.shallow_down(xs, i)
            new_x.append(F.pixel_shuffle(x_, 2**i))
        new_x = self.norm(self.mobivari(torch.cat(new_x, dim=1)))
        return new_x

    def flops(self, resolutions):
        H,W = resolutions
        flops = 0
        # self.shallow_down (iterative max-pool)
        flops += (H//2)*(W//2) * 2*2 * self.dim # shallow-down into stage2 output (iter1)
        flops += (H//2)*(W//2) * 2*2 * self.dim # shallow-down into stage3 output (iter1)
        flops += (H//4)*(W//4) * 2*2 * self.dim # shallow-down into stage3 output (iter2)
        flops += self.mobivari.flops((H,W))
        flops += self.norm.flops((H,W))
        return flops

class HRAMi(nn.Module):
    def __init__(self, dim, kernel_size=3, stride=1, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
        super(HRAMi, self).__init__()

        self.dim = dim
        self.kernel_size = kernel_size

        if mv_ver==1:
            self.mobivari = MobiVari1(dim+dim//4+dim//16+dim, kernel_size, stride, act=mv_act, out_dim=dim)
        elif mv_ver==2:
            self.mobivari = MobiVari2(dim+dim//4+dim//16+dim, kernel_size, stride, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
            # NAFBlock---------------------------------------------
        elif mv_ver==3:
            self.mobivari = NAFBlock(c=self.dim)
            # -------------------------------------------------------

    def forward(self, attn_list):
        for i, attn in enumerate(attn_list[:-1]):
            attn = F.pixel_shuffle(attn, 2**i)
            x = attn if i==0 else torch.cat([x, attn], dim=1)
        x = torch.cat([x, attn_list[-1]], dim=1)
        x = self.mobivari(x)
        return x

    def flops(self, resolutions):
        return self.mobivari.flops(resolutions)

class Reconstruction(nn.Module):
    def __init__(self, target_mode, dim, kernel_size=3, stride=1, num_mv=2, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
        super(Reconstruction, self).__init__()

        self.upscale = upscale = int(target_mode[-1]) if target_mode in ['light_x2', 'light_x3', 'light_x4'] else 1
        self.target_mode = target_mode
        self.dim = dim
        self.out_chans = 3 if 'gray' not in target_mode else 1
        self.kernel_size = kernel_size
        self.stride = stride
        self.num_mv = num_mv

        self.mobivari = nn.ModuleList()
        for i in range(num_mv):
            if mv_ver==3:
                self.mobivari.add_module(f'mobivari{i}', MobiVari1(dim, kernel_size, stride, mv_act))
            elif mv_ver==2:
                self.mobivari.add_module(f'mobivari{i}', MobiVari2(dim, kernel_size, stride, mv_act, None, exp_factor, expand_groups))
                # NAFBlock---------------------------------------------
            elif mv_ver==1:
                 self.mobivari.add_module(f'mobivari{i}',NAFBlock(c=self.dim))
                # -------------------------------------------------------


        self.conv = nn.Conv2d(dim, self.out_chans*(upscale**2), kernel_size, stride, kernel_size//2)
        self.shuffle = nn.PixelShuffle(upscale) if upscale!=1 else nn.Identity()
        self.final_conv = nn.Conv2d(self.out_chans, self.out_chans, kernel_size, stride, kernel_size//2)

    def forward(self, x):
        for mobivari in self.mobivari:
            x = mobivari(x)
        return self.final_conv(self.shuffle(self.conv(x)))

    def flops(self, resolutions):
        H,W = resolutions
        flops = 0
        for mobivari in self.mobivari: # self.mobivari
            flops += mobivari.flops((H,W))
        flops += H*W * self.kernel_size*self.kernel_size * self.dim * self.out_chans*(self.upscale**2) # self.conv
        flops += H*W * self.kernel_size*self.kernel_size * self.out_chans * self.out_chans # self.final_conv
        return flops

class RAMiT_FFAT(nn.Module):
    def __init__(self, in_chans=3, dim=64, depths=(6,4,4,6), num_heads=(4,4,4,4), head_dim=None, chsa_head_ratio=0.25, #depths = (6,4,4,6)
                 window_size=8, hidden_ratio=2.0, qkv_bias=True, mv_ver=1, exp_factor=1.2, expand_groups=4,
                 act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, tail_mv=2, weight_init='', target_mode='light_x2', img_norm=True,
                 attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True, mv_act=nn.LeakyReLU,
                 num_blocks = [1,4,6,6,8],
                 heads = [1,2,4,8],
                 ffn_expansion_factor = 2.66,
                 bias = False,
                 LayerNorm_type = 'WithBias',   ## Other option 'BiasFree'
                 FFAT = False, **kwargs):
        super(RAMiT_FFAT, self).__init__()
        self.cat_dim = sum([dim//(4**x) for x in range(len(depths)-1)]) # added to use the torchinfor.summary
        self.unit = 2**(len(depths)-2)*window_size

        self.in_chans = in_chans
        self.dim = dim
        self.depths = depths
        self.num_heads = num_heads
        self.head_dim = head_dim
        self.window_size = window_size
        self.hidden_ratio = hidden_ratio
        self.qkv_bias = qkv_bias
        self.act_layer = act_layer
        norm_layer = ReshapeLayerNorm if norm_layer == 'ReshapeLayerNorm' else norm_layer
        self.norm_layer = norm_layer = ReshapeLayerNorm if norm_layer == 'ReshapeLayerNorm' else norm_layer
        self.tail_mv = tail_mv

        self.scale = int(target_mode[-1]) if target_mode in ['light_x2', 'light_x3', 'light_x4'] else 1
        self.mean, self.std = mean_std(self.scale, target_mode)
        self.target_mode = target_mode
        self.img_norm = img_norm

        dpr = [x.item() for x in torch.linspace(0, drop_path, sum(depths))]  # stochastic depth decay rule

        self.shallow = ShallowModule(in_chans, dim, 3, 1)
        self.stage1 = EncoderStage(depths[1], dim, num_heads[0], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
                                   hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)


        # FFSANet
        self.cubic_11_1 = cubic_attention(dim//2, group=1, kernel=11)
        self.cubic_7_1 = cubic_attention(dim//2, group=1, kernel=7)

        self.cubic_11_2 = cubic_attention(dim//2, group=1, kernel=11)
        self.cubic_7_2 = cubic_attention(dim//2, group=1, kernel=7)

        self.cubic_11_3 = cubic_attention(dim//2, group=1, kernel=11)
        self.cubic_7_3 = cubic_attention(dim//2, group=1, kernel=7)
        # FFSANet


        self.down1 = Downsizing(dim, dim, norm_layer, mv_ver, mv_act)
        self.stage2 = EncoderStage(depths[1], dim, num_heads[1], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
                                   hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)
        self.down2 = Downsizing(dim, dim, norm_layer, mv_ver, mv_act)
        self.stage3 = EncoderStage(depths[1], dim, num_heads[2], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
                                   hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)





        if mv_ver==3:
            self.mobivari = MobiVari1(self.cat_dim, 3, 1, act=mv_act, out_dim=dim)
        elif mv_ver==2:
            self.mobivari = MobiVari2(self.cat_dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
            # NAFBlock not Accepted
            # NAFBlock---------------------------------------------
        elif mv_ver==1:
            self.mobivari = NAFBlock(c=self.dim)
            # -------------------------------------------------------
        self.act = act_layer()
        self.norm = norm_layer(dim)
        # self.bottleneck = Bottleneck(dim, len(depths)-1, act_layer, norm_layer, mv_ver, mv_act)

        self.stage4 = EncoderStage(depths[1], dim, num_heads[3], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
                                   hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)







        if mv_ver==3:
            self.mobivari_mix = MobiVari1(dim+dim//4+dim//16+dim, 3, 1, act=mv_act, out_dim=dim)
        elif mv_ver==2:
            self.mobivari_mix = MobiVari2(dim+dim//4+dim//16+dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
        # self.attn_mix = HRAMi(dim, 3, 1, mv_ver, mv_act)


        self.to_target = Reconstruction(target_mode, dim, 3, 1, tail_mv, mv_ver, mv_act, exp_factor, expand_groups)

        self.apply(self._init_weights)

    def forward_size_norm(self, x):
        _,_,h,w = x.size()
        padh = self.unit-(h%self.unit) if h%self.unit!=0 else 0
        padw = self.unit-(w%self.unit) if w%self.unit!=0 else 0
        x = TF.pad(x, (0,0,padw,padh))
        return TF.normalize(x, self.mean, self.std) if self.img_norm else x

    def forward(self, x):
        _,_,H_ori,W_ori = x.size()
        x = self.forward_size_norm(x)
        B, C, H, W = x.size()
        x_lq = x if self.scale == 1 else None # due to residual image (except for only SR)

        shallow = self.shallow(x)
        o1_, attn1 = self.stage1(shallow) # [B, C, H, W]


        # FFAT
        out = torch.chunk(o1_, 2, dim=1)
        out_11_1 = self.cubic_11_1(out[0])
        out_7_1 = self.cubic_7_1(out[1])
        o1_ = torch.cat((out_11_1, out_7_1), dim=1)
        # FFAT


        o1 = self.down1(o1_) # [B, C, H//2, W//2]

        o2_, attn2 = self.stage2(o1) # [B, C, H//2, W//2]


        # FFAT
        out = torch.chunk(o2_, 2, dim=1)
        out_11_2 = self.cubic_11_2(out[0])
        out_7_2 = self.cubic_7_2(out[1])
        o2_ = torch.cat((out_11_2, out_7_2), dim=1)
        # FFAT


        o2 = self.down2(o2_) # [B, C, H//4, W//4]

        o3_, attn3 = self.stage3(o2) # [B, C, H//4, W//4]


        # FFAT
        out = torch.chunk(o3_, 2, dim=1)
        out_11_3 = self.cubic_11_3(out[0])
        out_7_3 = self.cubic_7_3(out[1])
        o3_ = torch.cat((out_11_3, out_7_3), dim=1)
        # FFAT




# Bottleneck
        def shallow_down( x, r):
          for _ in range(r):
              x = F.max_pool2d(x, 2)
          return F.leaky_relu(x)
        x_list = [shallow, o1_, o2_, o3_]
        xs = x_list[0]
        new_x = []
        for i in range(len(x_list[1:])):
            x_ = x_list[i+1]+shallow_down(xs, i)
            new_x.append(F.pixel_shuffle(x_, 2**i))
        ob = self.norm(self.mobivari(torch.cat(new_x, dim=1)))
        # ob = self.bottleneck([shallow, o1_, o2_, o3_]) # [B, C, H, W]



        o4, attn4 = self.stage4(ob) # [B, C, H, W]






# HRAMiT
        attn_list = [attn1, attn2, attn3, attn4]
        for i, attn in enumerate(attn_list[:-1]):
            attn = F.pixel_shuffle(attn, 2**i)
            x = attn if i==0 else torch.cat([x, attn], dim=1)
        x = torch.cat([x, attn_list[-1]], dim=1)
        mix = self.mobivari_mix(x)
        # mix = self.attn_mix([attn1, attn2, attn3, attn4]) # [B, C, H, W]



        o4 = o4 * mix # [B, C, H, W]

        out = self.to_target(o4+shallow) # global skip connection
        out = out + x_lq if self.scale == 1 else out # residual output (except for only SR)

        out = denormalize(out, self.mean, self.std) if self.img_norm else out
        out = out[:, :, :H_ori*self.scale, :W_ori*self.scale]
        return out

    def flops(self, resolutions):
        H_ori,W_ori = resolutions
        padh = self.unit-(H_ori%self.unit) if H_ori%self.unit!=0 else 0
        padw = self.unit-(W_ori%self.unit) if W_ori%self.unit!=0 else 0
        H,W = H_ori+padh,W_ori+padw
        flops = 0
        flops += self.shallow.flops((H,W))
        flops += self.stage1.flops((H,W))
        flops += self.down1.flops((H,W))
        flops += self.stage2.flops((H//2,W//2))
        flops += self.down2.flops((H//2,W//2))
        flops += self.stage3.flops((H//4,W//4))
        flops += self.bottleneck.flops((H,W))
        flops += self.stage4.flops((H,W))
        flops += self.attn_mix.flops((H,W))
        flops += self.dim * H*W # o4 = o4*mix
        flops += self.to_target.flops((H,W))
        return flops

    def _init_weights(self, m):
        # Swin V2 manner
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        nwd = set()
        for n, _ in self.named_parameters():
            if 'relative_position_bias_table' in n:
                nwd.add(n)
        return nwd






# ----------------------------Shadow_R-------------------------
    # @torch.no_grad()
    # def _init_weights(module_list, scale=1, bias_fill=0, **kwargs):
    #   """Initialize network weights.

    #   Args:
    #       module_list (list[nn.Module] | nn.Module): Modules to be initialized.
    #       scale (float): Scale initialized weights, especially for residual
    #           blocks. Default: 1.
    #       bias_fill (float): The value to fill bias. Default: 0
    #       kwargs (dict): Other arguments for initialization function.
    #   """
    #   if not isinstance(module_list, list):
    #       module_list = [module_list]
    #   for module in module_list:
    #       for m in module.modules():
    #           if isinstance(m, nn.Conv2d):
    #               nn.init.kaiming_normal_(m.weight, **kwargs)
    #               m.weight.data *= scale
    #               if m.bias is not None:
    #                   m.bias.data.fill_(bias_fill)
    #           elif isinstance(m, nn.Linear):
    #               nn.init.kaiming_normal_(m.weight, **kwargs)
    #               m.weight.data *= scale
    #               if m.bias is not None:
    #                   m.bias.data.fill_(bias_fill)
    #           elif isinstance(m, _BatchNorm):
    #               nn.init.constant_(m.weight, 1)
    #               if m.bias is not None:
    #                   m.bias.data.fill_(bias_fill)

/usr/local/lib/python3.10/dist-packages/timm/models/fx_features.py:4: FutureWarning: Importing from timm.models.fx_features is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [7]:
G1 = RAMiT_FFAT(target_mode='light_realdn',img_norm=False, mv_ver = 3)
G1 = nn.DataParallel(G1)
G1.to('cuda')
def count_parameters(model):
    return sum(p.numel() for p in G1.parameters() if p.requires_grad)

# Create the model

# Print the number of parameters
print(f'The model has {count_parameters(G1):,} trainable parameters')

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


The model has 672,151 trainable parameters


In [8]:
# # Load the checkpoint
# checkpoint = torch.load(r'C:\Users\Admin\Desktop\AI_RAMiT\FFAT_RAMiT\SBSR70.pth', map_location='cuda')

# # Remove "module." prefix if it exists in the keys
# from collections import OrderedDict
# new_state_dict = OrderedDict()
# for k, v in checkpoint.items():
#     name = k[7:] if k.startswith("module.") else k  # remove "module." prefix
#     new_state_dict[name] = v

# # Load the modified state_dict into the model
# G1 = RAMiT_FFAT(target_mode='light_realdn', img_norm=False, mv_ver=3)
# G1.load_state_dict(new_state_dict)

# # Apply DataParallel and move to GPU
# G1 = nn.DataParallel(G1)
# G1.to('cuda')


In [9]:
from torchinfo import summary
summary(G1, (1, 3, 360, 480))

Layer (type:depth-idx)                                  Output Shape              Param #
DataParallel                                            [1, 3, 360, 480]          --
├─RAMiT_FFAT: 1-1                                       [1, 3, 360, 480]          --
│    └─ShallowModule: 2-1                               [1, 64, 384, 480]         --
│    │    └─Conv2d: 3-1                                 [1, 64, 384, 480]         1,792
│    └─EncoderStage: 2-2                                [1, 64, 384, 480]         --
│    │    └─ModuleList: 3-2                             --                        149,660
│    └─cubic_attention: 2-3                             [1, 32, 384, 480]         64
│    │    └─spatial_strip_att: 3-3                      [1, 32, 384, 480]         352
│    │    └─spatial_strip_att: 3-4                      [1, 32, 384, 480]         352
│    │    └─spatial_strip_att: 3-5                      [1, 32, 384, 480]         (recursive)
│    │    └─spatial_strip_att: 3-6       

<!-- ## FFAT + RAMiT + NAFNet -->

## Error RAMiT

In [10]:
# # RAMiT (Reciprocal Attention Mixing Transformer)

# from pdb import set_trace as stx
# import numbers






# from torch import nn, Tensor
# from einops import rearrange
# import torch.nn.functional as F
# import torch
# from torchvision.transforms import functional as TF
# import math
# from RAMiT.my_model.common.mean_std import mean_std
# from RAMiT.utils.etc_utils import denormalize
# from timm.models.fx_features import register_notrace_function
# from timm.models.layers import trunc_normal_, to_2tuple
# from torch.nn.modules.batchnorm import _BatchNorm
# def make_model(args, opts, pe):
#     model = RAMiT(target_mode=args.target_mode,
#                   img_norm=args.img_norm,
#                   in_chans=opts['in_chans'],
#                   dim = opts['dim'],
#                   depths = opts['depths'],
#                   num_heads = opts['num_heads'],
#                   head_dim = opts['head_dim'],
#                   chsa_head_ratio = opts['chsa_head_ratio'],
#                   window_size = opts['window_size'],
#                   hidden_ratio = opts['hidden_ratio'],
#                   qkv_bias = opts['qkv_bias'],
#                   mv_ver = opts['mv_ver'],
#                   exp_factor=opts['exp_factor'],
#                   expand_groups=opts['expand_groups'],
#                   act_layer = opts['act_layer'],
#                   norm_layer = opts['norm_layer'],
#                   tail_mv = opts['tail_mv'],
#                   attn_drop = opts['attn_drop'],
#                   proj_drop = opts['proj_drop'],
#                   drop_path = opts['drop_path'],
#                   helper = opts['helper'],
#                   mv_act = opts['mv_act']
#                  )
#     num_params = sum([p.numel() for n,p in model.named_parameters()])
#     print(format(num_params, ','))
#     return model
























# # ---------------------------------------------------------------------------Shadow_R---------------------------------------------------------------------------

# def to_3d(x):
#     return rearrange(x, 'b c h w -> b (h w) c')

# def to_4d(x,h,w):
#     return rearrange(x, 'b (h w) c -> b c h w',h=h,w=w)

# class BiasFree_LayerNorm(nn.Module):
#     def __init__(self, normalized_shape):
#         super(BiasFree_LayerNorm, self).__init__()
#         if isinstance(normalized_shape, numbers.Integral):
#             normalized_shape = (normalized_shape,)
#         normalized_shape = torch.Size(normalized_shape)

#         assert len(normalized_shape) == 1

#         self.weight = nn.Parameter(torch.ones(normalized_shape))
#         self.normalized_shape = normalized_shape

#     def forward(self, x):
#         sigma = x.var(-1, keepdim=True, unbiased=False)
#         return x / torch.sqrt(sigma+1e-5) * self.weight

# class WithBias_LayerNorm(nn.Module):
#     def __init__(self, normalized_shape):
#         super(WithBias_LayerNorm, self).__init__()
#         if isinstance(normalized_shape, numbers.Integral):
#             normalized_shape = (normalized_shape,)
#         normalized_shape = torch.Size(normalized_shape)

#         assert len(normalized_shape) == 1

#         self.weight = nn.Parameter(torch.ones(normalized_shape))
#         self.bias = nn.Parameter(torch.zeros(normalized_shape))
#         self.normalized_shape = normalized_shape

#     def forward(self, x):
#         mu = x.mean(-1, keepdim=True)
#         sigma = x.var(-1, keepdim=True, unbiased=False)
#         return (x - mu) / torch.sqrt(sigma+1e-5) * self.weight + self.bias


# class LayerNorm(nn.Module):
#     def __init__(self, dim, LayerNorm_type):
#         super(LayerNorm, self).__init__()
#         if LayerNorm_type =='BiasFree':
#             self.body = BiasFree_LayerNorm(dim)
#         else:
#             self.body = WithBias_LayerNorm(dim)

#     def forward(self, x):
#         h, w = x.shape[-2:]
#         return to_4d(self.body(to_3d(x)), h, w)



# ##########################################################################
# ## Gated-Dconv Feed-Forward Network (GDFN)
# class FeedForwardFFAT(nn.Module):
#     def __init__(self, dim, ffn_expansion_factor, bias):
#         super(FeedForwardFFAT, self).__init__()

#         hidden_features = int(dim*ffn_expansion_factor)

#         self.project_in = nn.Conv2d(dim, hidden_features*2, kernel_size=1, bias=bias)

#         self.dwconv = nn.Conv2d(hidden_features*2, hidden_features*2, kernel_size=3, stride=1, padding=1, groups=hidden_features*2, bias=bias)

#         self.project_out = nn.Conv2d(hidden_features, dim, kernel_size=1, bias=bias)

#     def forward(self, x):
#         x = self.project_in(x)
#         x1, x2 = self.dwconv(x).chunk(2, dim=1)
#         x = F.gelu(x1) * x2
#         x = self.project_out(x)
#         return x



# class Attention(nn.Module):
#     def __init__(self, dim, num_heads, bias):
#         super(Attention, self).__init__()

#         self.num_heads = num_heads

#         self.to_hidden = nn.Conv2d(dim, dim * 6, kernel_size=1, bias=bias)
#         self.to_hidden_dw = nn.Conv2d(dim * 6, dim * 6, kernel_size=3, stride=1, padding=1, groups=dim * 6, bias=bias)

#         self.project_out = nn.Conv2d(dim * 2, dim, kernel_size=1, bias=bias)

#         self.norm = LayerNorm(dim * 2, LayerNorm_type='WithBias')

#         self.patch_size = 2 # 8

#     def forward(self, x):
#         hidden = self.to_hidden(x)

#         q, k, v = self.to_hidden_dw(hidden).chunk(3, dim=1)

#         q_patch = rearrange(q, 'b c (h patch1) (w patch2) -> b c h w patch1 patch2', patch1=self.patch_size,
#                             patch2=self.patch_size)
#         k_patch = rearrange(k, 'b c (h patch1) (w patch2) -> b c h w patch1 patch2', patch1=self.patch_size,
#                             patch2=self.patch_size)
#         q_fft = torch.fft.rfft2(q_patch.float())
#         k_fft = torch.fft.rfft2(k_patch.float())

#         out = q_fft * k_fft
#         out = torch.fft.irfft2(out, s=(self.patch_size, self.patch_size))
#         out = rearrange(out, 'b c h w patch1 patch2 -> b c (h patch1) (w patch2)', patch1=self.patch_size,
#                         patch2=self.patch_size)

#         out = self.norm(out)

#         output = v * out
#         output = self.project_out(output)

#         return output






# ##########################################################################
# class TransformerBlock(nn.Module):
#     def __init__(self, dim, num_heads, ffn_expansion_factor, bias, LayerNorm_type):
#         super(TransformerBlock, self).__init__()

#         self.norm1 = LayerNorm(dim, LayerNorm_type)
#         self.attn = Attention(dim, num_heads, bias)
#         self.norm2 = LayerNorm(dim, LayerNorm_type)
#         self.ffn = FeedForwardFFAT(dim, ffn_expansion_factor, bias)

#     def forward(self, x):
#         x = x + self.attn(self.norm1(x))
#         x = x + self.ffn(self.norm2(x))

#         return x
# # ---------------------------------------------------------------------------Shadow_R---------------------------------------------------------------------------













# class ShallowModule(nn.Module):
#     def __init__(self, in_chans, out_chans, kernel_size=3, stride=1):
#         super(ShallowModule, self).__init__()

#         self.in_chans = in_chans
#         self.out_chans = out_chans
#         self.kernel_size = kernel_size
#         self.stride = stride

#         self.conv = nn.Conv2d(in_chans, out_chans, kernel_size, stride, kernel_size//2)

#     def forward(self, x):
#         return self.conv(x)

#     def flops(self, resolutions):
#         return resolutions[0]*resolutions[1] * self.kernel_size*self.kernel_size * self.in_chans * self.out_chans

# class QKVProjection(nn.Module):
#     def __init__(self, dim, num_head, qkv_bias=True):
#         super(QKVProjection, self).__init__()
#         self.dim = dim
#         self.num_head = num_head

#         self.qkv = nn.Conv2d(dim, 3*dim, 1, bias=qkv_bias)

#     def forward(self, x):
#         B, C, H, W = x.size()
#         qkv = self.qkv(x)
#         qkv = rearrange(qkv, 'b (l c) h w -> b l c h w', l=self.num_head)
#         return qkv

#     def flops(self, resolutions):
#         return resolutions[0]*resolutions[1] * 1*1 * self.dim * 3*self.dim

# def get_relative_position_index(win_h, win_w):
#     # get pair-wise relative position index for each token inside the window
#     coords = torch.stack(torch.meshgrid([torch.arange(win_h), torch.arange(win_w)]))  # 2, Wh, Ww
#     coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
#     relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww (xaxis matrix & yaxis matrix)
#     relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
#     relative_coords[:, :, 0] += win_h - 1  # shift to start from 0
#     relative_coords[:, :, 1] += win_w - 1
#     relative_coords[:, :, 0] *= 2 * win_w - 1
#     return relative_coords.sum(-1)  # Wh*Ww, Wh*Ww

# class SpatialSelfAttention(nn.Module):
#     def __init__(self, dim, num_head, total_head, window_size=8, shift=0, attn_drop=0.0, proj_drop=0.0, helper=True):
#         super(SpatialSelfAttention, self).__init__()
#         self.dim = dim
#         self.num_head = num_head
#         self.total_head = total_head
#         self.window_size = window_size
#         self.window_area = window_size**2
#         self.shift = shift
#         self.helper = helper

#         self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_head, 1, 1))), requires_grad=True)

#         # define a parameter table of relative position bias, shape: 2*Wh-1 * 2*Ww-1, nH
#         self.relative_position_bias_table = nn.Parameter(torch.zeros((2 * window_size - 1) * (2 * window_size - 1), num_head))

#         # get pair-wise relative position index for each token inside the window
#         self.register_buffer("relative_position_index", get_relative_position_index(window_size, window_size))

#         self.attn_drop = nn.Dropout(attn_drop)

#         self.proj = nn.Conv2d(dim*num_head, dim*num_head, 1)
#         self.proj_drop = nn.Dropout(proj_drop)

#     def _get_rel_pos_bias(self) -> torch.Tensor:
#         relative_position_bias = self.relative_position_bias_table[
#             self.relative_position_index.view(-1)].view(self.window_area, self.window_area, -1)  # Wh*Ww,Wh*Ww,nH
#         relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
#         return relative_position_bias.unsqueeze(0)

#     def forward(self, qkv, ch=None):
#         B, L, C, H, W = qkv.size()
#         # window shift
#         if self.shift > 0:
#             qkv = torch.roll(qkv, shifts=(-self.shift, -self.shift), dims=(-2,-1))

#         # window partition
#         q,k,v = rearrange(qkv, 'b l c (h wh) (w ww) -> (b h w) l (wh ww) c',
#                           wh=self.window_size, ww=self.window_size).chunk(3, dim=-1) # [B_, L1, hw, C/L] respectively
#         if ch is not None and self.helper: # [B, C, H, W]
#             if self.shift > 0:
#                 ch = torch.roll(ch, shifts=(-self.shift, -self.shift), dims=(-2,-1))
#             ch = rearrange(ch, 'b (l c) (h wh) (w ww) -> (b h w) l (wh ww) c',
#                            l=self.total_head-self.num_head, wh=self.window_size, ww=self.window_size) # [B_, L1, hw, C/L]
#             ch = torch.mean(ch, dim=1, keepdim=True) # head squeeze [B_, 1, hw, C/L]
#             v = v*ch # [B_, L1, hw, C/L]

#         attn = F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(2,-1) # [B_, L1, hw, hw]
#         logit_scale = torch.clamp(self.logit_scale, max=math.log(1. / 0.01)).exp()
#         attn = attn * logit_scale

#         attn = attn + self._get_rel_pos_bias()
#         attn = self.attn_drop(F.softmax(attn, dim=-1))

#         x = attn @ v # [B_, L1, hw, C/L]

#         # window unpartition + head merge
#         x = window_unpartition(x, (H,W), self.window_size) # [B, L1*C/L, H, W]
#         x = self.proj_drop(self.proj(x))

#         # window reverse shift
#         if self.shift > 0:
#             x = torch.roll(x, shifts=(self.shift, self.shift), dims=(-2,-1))

#         return x

#     def flops(self, resolutions):
#         H,W = resolutions
#         num_wins = H//self.window_size * W//self.window_size
#         flops = self.num_head * H*W * self.dim if self.helper else 0 # v = v*ch
#         flops += num_wins * self.num_head * self.window_area * self.dim * self.window_area # attn = Q@K^T
#         flops += num_wins * self.num_head * self.window_area * self.window_area * self.dim # attn@V
#         flops += H*W * 1*1 * self.num_head*self.dim * self.num_head*self.dim # self.proj
#         return flops

# @register_notrace_function
# def window_unpartition(x, resolutions, window_size):
#     return rearrange(x, '(b h w) l (wh ww) c -> b (l c) (h wh) (w ww)',
#                      h=resolutions[0]//window_size, w=resolutions[1]//window_size, wh=window_size)

# class ChannelSelfAttention(nn.Module):
#     def __init__(self, dim, num_head, total_head, attn_drop=0.0, proj_drop=0.0, helper=True):
#         super(ChannelSelfAttention, self).__init__()
#         self.dim = dim
#         self.num_head = num_head
#         self.total_head = total_head
#         self.helper = helper

#         self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_head, 1, 1))), requires_grad=True)

#         self.attn_drop = nn.Dropout(attn_drop)

#         self.proj = nn.Conv2d(dim*num_head, dim*num_head, 1)
#         self.proj_drop = nn.Dropout(proj_drop)

#     def forward(self, qkv, sp=None):
#         B, L, C, H, W = qkv.size()

#         q,k,v = rearrange(qkv, 'b l c h w -> b l c (h w)').chunk(3, dim=-2) # [B, L2, C/L, HW]
#         if sp is not None and self.helper:
#             sp = torch.mean(sp, dim=1, keepdim=True) # channel squeeze # [B, 1, H, W]
#             sp = rearrange(sp, 'b (l c) h w -> b l c (h w)', l=1) # [B, 1, 1, HW]
#             v = v*sp # [B, L2, C/L, HW]

#         attn = F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(2,-1) # [B, L2, C/L, C/L]
#         logit_scale = torch.clamp(self.logit_scale, max=math.log(1. / 0.01)).exp()
#         attn = attn * logit_scale

#         attn = F.softmax(attn, dim=-1)
#         attn = self.attn_drop(attn)

#         x = attn @ v # [B, L2, C/L, HW]

#         # head merge
#         x = rearrange(x, 'b l c (h w) -> b (l c) h w', h=H) # [B, L2*C/L, H, W]
#         x = self.proj_drop(self.proj(x)) # [B, L2*C/L, H, W]

#         return x

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = self.num_head * self.dim * H*W if self.helper else 0 # v = v*sp
#         flops += self.num_head * self.dim * H*W * self.dim # attn = Q@K^T
#         flops += self.num_head * self.dim * self.dim * H*W # attn@V
#         flops += H*W * 1*1 * self.num_head*self.dim * self.num_head*self.dim # self.proj
#         return flops

# class ReshapeLayerNorm(nn.Module):
#     def __init__(self, dim, norm_layer=nn.LayerNorm):
#         super(ReshapeLayerNorm, self).__init__()

#         self.dim = dim
#         self.norm = norm_layer(dim)

#     def forward(self, x):
#         B, C, H, W = x.size()
#         x = rearrange(x, 'b c h w -> b (h w) c')
#         x = self.norm(x)
#         x = rearrange(x, 'b (h w) c -> b c h w', h=H)
#         return x

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = 0
#         flops += H*W * self.dim
#         return flops

# class MobiVari1(nn.Module): # MobileNet v1 Variants
#     def __init__(self, dim, kernel_size, stride, act=nn.LeakyReLU, out_dim=None):
#         super(MobiVari1, self).__init__()
#         self.dim = dim
#         self.kernel_size = kernel_size
#         self.out_dim = out_dim or dim

#         self.dw_conv = nn.Conv2d(dim, dim, kernel_size, stride, kernel_size//2, groups=dim)
#         self.pw_conv = nn.Conv2d(dim, self.out_dim, 1, 1, 0)
#         self.act = act()

#     def forward(self, x):
#         out = self.act(self.pw_conv(self.act(self.dw_conv(x))+x))
#         return out + x if self.dim==self.out_dim else out

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = H*W * self.kernel_size*self.kernel_size * self.dim  +  H*W * 1*1 * self.dim * self.out_dim # self.dw_conv + self.pw_conv
#         return flops

# class MobiVari2(MobiVari1): # MobileNet v2 Variants
#     def __init__(self, dim, kernel_size, stride, act=nn.LeakyReLU, out_dim=None, exp_factor=1.2, expand_groups=4):
#         super(MobiVari2, self).__init__(dim, kernel_size, stride, act, out_dim)
#         self.expand_groups = expand_groups
#         expand_dim = int(dim*exp_factor)
#         expand_dim = expand_dim+(expand_groups-expand_dim%expand_groups)
#         self.expand_dim = expand_dim

#         self.exp_conv = nn.Conv2d(dim, self.expand_dim, 1, 1, 0, groups=expand_groups)
#         self.dw_conv = nn.Conv2d(expand_dim, expand_dim, kernel_size, stride, kernel_size//2, groups=expand_dim)
#         self.pw_conv = nn.Conv2d(expand_dim, self.out_dim, 1, 1, 0)

#     def forward(self, x):
#         x1 = self.act(self.exp_conv(x))
#         out = self.pw_conv(self.act(self.dw_conv(x1)+x1))
#         return out + x if self.dim==self.out_dim else out

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = H*W * 1*1 * (self.dim//self.expand_groups) * self.expand_dim # self.exp_conv
#         flops += H*W * self.kernel_size*self.kernel_size * self.expand_dim # self.dw_conv
#         flops += H*W * 1*1 * self.expand_dim * self.out_dim # self.pw_conv
#         return flops

# class FeedForward(nn.Module):
#     def __init__(self, dim, hidden_ratio, act_layer=nn.GELU, bias=True, drop=0.0):
#         super(FeedForward, self).__init__()

#         self.dim = dim
#         self.hidden_ratio = hidden_ratio

#         self.hidden = nn.Conv2d(dim, int(dim*hidden_ratio), 1, bias=bias)
#         self.drop1 = nn.Dropout(drop)
#         self.out = nn.Conv2d(int(dim*hidden_ratio), dim, 1, bias=bias)
#         self.drop2 = nn.Dropout(drop)
#         self.act = act_layer()

#     def forward(self, x):
#         return self.drop2(self.out(self.drop1(self.act(self.hidden(x)))))

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = 2 * H*W * 1*1 * self.dim * self.dim*self.hidden_ratio # self.hidden + self.out
#         return flops

# class NoLayer(nn.Identity):
#     def __init__(self):
#         super(NoLayer, self).__init__()
#     def flops(self, resolutions):
#         return 0
#     def forward(self, x, **kwargs):
#         return x.flatten(1,2)

# class DRAMiTransformer(nn.Module): # Reciprocal Attention Transformer Block
#     def __init__(self, dim, num_head, chsa_head_ratio, FFAT, window_size=8, shift=0, head_dim=None, qkv_bias=True, mv_ver=1,
#                  hidden_ratio=2.0, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True,
#                  mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4,
#                  num_blocks = [4,6,6,8], heads = [1,2,4,8], ffn_expansion_factor = 2.66, bias = False, LayerNorm_type = 'WithBias'):
#         super(DRAMiTransformer, self).__init__()

#         self.dim = dim
#         self.num_head = num_head
#         self.window_size = window_size
#         self.chsa_head = int(num_head*chsa_head_ratio)
#         self.shift = shift
#         self.helper = helper
#         self.FFAT = FFAT

#         self.qkv_proj = QKVProjection(dim, num_head, qkv_bias=qkv_bias)
#         self.sp_attn = SpatialSelfAttention(dim//num_head, num_head-self.chsa_head, num_head,
#                                             window_size, shift, attn_drop, proj_drop, helper) if num_head-self.chsa_head != 0 else NoLayer()
#         self.ch_attn = ChannelSelfAttention(dim//num_head, self.chsa_head, num_head, attn_drop, proj_drop, helper) if self.chsa_head != 0 else NoLayer()


#         #FFAT
#         if self.FFAT:
#           self.encoder_level1 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])
#         #FFAT


#         if mv_ver==1:
#             self.mobivari = MobiVari1(dim, 3, 1, act=mv_act)
#         elif mv_ver==2:
#             self.mobivari = MobiVari2(dim, 3, 1, act=mv_act, out_dim=None, exp_factor=exp_factor, expand_groups=expand_groups)

#             # NAFBlock---------------------------------------------
#         elif mv_ver==3:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------

#         self.norm1 = norm_layer(dim)

#         self.ffn = FeedForward(dim, hidden_ratio, act_layer=act_layer)
#         self.norm2 = norm_layer(dim)

#         self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

#     def forward(self, x, sp_=None, ch_=None):
#         B, C, H, W = x.size()

#         # QKV projection + head split
#         qkv = self.qkv_proj(x) # [B, L, C, H, W]

#         # SP-SA / CH-SA
#         sp = self.sp_attn(qkv[:,:self.num_head-self.chsa_head], ch=ch_) # [B, L1*C/L, H, W]
#         ch = self.ch_attn(qkv[:,self.num_head-self.chsa_head:], sp=sp_) # [B, L2*C/L, H, W]


#         #FFAT
#         if self.FFAT:
#           FFAT_block = self.encoder_level1(torch.cat([sp, ch], dim=1))
#           attn0 = self.mobivari(FFAT_block)
#         #FFAT


#         else:
#           attn0 = self.mobivari(torch.cat([sp, ch], dim=1)) # merge [B, C, H, W]
#         attn = self.drop_path(self.norm1(attn0)) + x # LN, skip connection [B, C, H, W]

#         # FFN
#         out = self.drop_path(self.norm2(self.ffn(attn))) + attn # FFN, LN, skip connection [B, C, H, W]

#         return out, sp, ch, attn0

#     def flops(self, resolutions):
#         flops = self.qkv_proj.flops(resolutions)
#         flops += self.sp_attn.flops(resolutions)
#         flops += self.ch_attn.flops(resolutions)
#         flops += self.mobivari.flops(resolutions)
#         flops += self.norm1.flops(resolutions)
#         flops += self.ffn.flops(resolutions)
#         flops += self.norm2.flops(resolutions)
#         params = sum([p.numel() for n,p in self.named_parameters()])
#         return flops

# class EncoderStage(nn.Module):
#     def __init__(self, depth, dim, num_head, chsa_head_ratio, FFAT, window_size=8, head_dim=None,
#                  qkv_bias=True, mv_ver=1, hidden_ratio=2.0, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm,
#                  attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True, mv_act=nn.LeakyReLU,):
#         super(EncoderStage, self).__init__()

#         self.depth = depth
#         self.dim = dim
#         self.num_head = num_head
#         self.window_size = window_size
#         shift = window_size//2

#         self.blocks = nn.ModuleList()
#         for d in range(depth):
#             self.blocks.add_module(f'block{d}', DRAMiTransformer(dim, num_head, chsa_head_ratio, FFAT,
#                                                                  window_size, 0 if d%2==0 else shift, head_dim, qkv_bias,
#                                                                  mv_ver, hidden_ratio, act_layer, norm_layer,
#                                                                  attn_drop, proj_drop, drop_path, helper, mv_act,
#                                                                  ))
#     def forward(self, x):
#         sp, ch = None, None
#         for i, blk in enumerate(self.blocks):
#             x, sp, ch, attn = blk(x, sp, ch)
#             sp, ch = (None, None) if (sp.size(1)==0 or ch.size(1)==0) else (sp, ch) # pure SPSA or CHSA
#         return x, attn

#     def flops(self, resolutions):
#         flops = 0
#         for blk in self.blocks:
#             flops += blk.flops(resolutions)
#         return flops

# class Downsizing(nn.Module):
#     """ Patch Merging Layer.

#     Args:
#         dim (int): Number of input dimension (channels).
#         downsample_dim (int, optional): Number of output dimension (channels) (dim if not set).  Default: None
#         norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
#     """

#     def __init__(self, dim, downsample_dim=None, norm_layer=ReshapeLayerNorm, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
#         super(Downsizing, self).__init__()
#         self.dim = dim
#         self.downsample_dim = downsample_dim or dim
#         self.norm = norm_layer(4*dim)
#         if mv_ver==1:
#             self.reduction = MobiVari1(4*dim, 3, 1, act=mv_act, out_dim=self.downsample_dim)
#         elif mv_ver==2:
#             self.reduction = MobiVari2(4*dim, 3, 1, act=mv_act, out_dim=self.downsample_dim, exp_factor=exp_factor, expand_groups=expand_groups)

#             # NAFBlock---------------------------------------------
#         elif mv_ver==3:
#             self.reduction = NAFBlock(c=4*dim)
#             # -------------------------------------------------------

#     def forward(self, x):
#         B, C, H, W = x.size()

#         # Concat 2x2
#         x0 = x[:, :, 0::2, 0::2]  # [B, C, H/2, W/2], top-left
#         x1 = x[:, :, 0::2, 1::2]  # [B, C, H/2, W/2], top-right
#         x2 = x[:, :, 1::2, 0::2]  # [B, C, H/2, W/2], bottom-left
#         x3 = x[:, :, 1::2, 1::2]  # [B, C, H/2, W/2], bottom-right
#         x = torch.cat([x0, x1, x2, x3], dim=1)  # [B, 4C, H/2, W/2]
#         return self.reduction(self.norm(x)) # [B, C, H/2, W/2]

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = self.norm.flops((H//2,W//2)) + self.reduction.flops((H//2,W//2))
#         return flops

# class Bottleneck(nn.Module):
#     def __init__(self, dim, num_stages, act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
#         super(Bottleneck, self).__init__()

#         self.dim = dim
#         self.cat_dim = sum([dim//(4**x) for x in range(num_stages)])
#         if mv_ver==1:
#             self.mobivari = MobiVari1(self.cat_dim, 3, 1, act=mv_act, out_dim=dim)
#         elif mv_ver==2:
#             self.mobivari = MobiVari2(self.cat_dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)

#             # NAFBlock---------------------------------------------
#         elif mv_ver==3:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------
#         self.act = act_layer()
#         self.norm = norm_layer(dim)

#     def shallow_down(self, x, r):
#         for _ in range(r):
#             x = F.max_pool2d(x, 2)
#         return F.leaky_relu(x)

#     def forward(self, x_list):
#         xs = x_list[0]
#         new_x = []
#         for i in range(len(x_list[1:])):
#             x_ = x_list[i+1]+self.shallow_down(xs, i)
#             new_x.append(F.pixel_shuffle(x_, 2**i))
#         new_x = self.norm(self.mobivari(torch.cat(new_x, dim=1)))
#         return new_x

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = 0
#         # self.shallow_down (iterative max-pool)
#         flops += (H//2)*(W//2) * 2*2 * self.dim # shallow-down into stage2 output (iter1)
#         flops += (H//2)*(W//2) * 2*2 * self.dim # shallow-down into stage3 output (iter1)
#         flops += (H//4)*(W//4) * 2*2 * self.dim # shallow-down into stage3 output (iter2)
#         flops += self.mobivari.flops((H,W))
#         flops += self.norm.flops((H,W))
#         return flops

# class HRAMi(nn.Module):
#     def __init__(self, dim, kernel_size=3, stride=1, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
#         super(HRAMi, self).__init__()

#         self.dim = dim
#         self.kernel_size = kernel_size

#         if mv_ver==1:
#             self.mobivari = MobiVari1(dim+dim//4+dim//16+dim, kernel_size, stride, act=mv_act, out_dim=dim)
#         elif mv_ver==2:
#             self.mobivari = MobiVari2(dim+dim//4+dim//16+dim, kernel_size, stride, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
#             # NAFBlock---------------------------------------------
#         elif mv_ver==3:
#             self.mobivari = NAFBlock(c=self.dim)
#             # -------------------------------------------------------

#     def forward(self, attn_list):
#         for i, attn in enumerate(attn_list[:-1]):
#             attn = F.pixel_shuffle(attn, 2**i)
#             x = attn if i==0 else torch.cat([x, attn], dim=1)
#         x = torch.cat([x, attn_list[-1]], dim=1)
#         x = self.mobivari(x)
#         return x

#     def flops(self, resolutions):
#         return self.mobivari.flops(resolutions)

# class Reconstruction(nn.Module):
#     def __init__(self, target_mode, dim, kernel_size=3, stride=1, num_mv=2, mv_ver=1, mv_act=nn.LeakyReLU, exp_factor=1.2, expand_groups=4):
#         super(Reconstruction, self).__init__()

#         self.upscale = upscale = int(target_mode[-1]) if target_mode in ['light_x2', 'light_x3', 'light_x4'] else 1
#         self.target_mode = target_mode
#         self.dim = dim
#         self.out_chans = 3 if 'gray' not in target_mode else 1
#         self.kernel_size = kernel_size
#         self.stride = stride
#         self.num_mv = num_mv

#         self.mobivari = nn.ModuleList()
#         for i in range(num_mv):
#             if mv_ver==1:
#                 self.mobivari.add_module(f'mobivari{i}', MobiVari1(dim, kernel_size, stride, mv_act))
#             elif mv_ver==2:
#                 self.mobivari.add_module(f'mobivari{i}', MobiVari2(dim, kernel_size, stride, mv_act, None, exp_factor, expand_groups))
#                 # NAFBlock---------------------------------------------
#             elif mv_ver==3:
#                 self.mobivari.add_module(f'mobivari{i}', NAFBlock(c=self.dim))
#                 # -------------------------------------------------------


#         self.conv = nn.Conv2d(dim, self.out_chans*(upscale**2), kernel_size, stride, kernel_size//2)
#         self.shuffle = nn.PixelShuffle(upscale) if upscale!=1 else nn.Identity()
#         self.final_conv = nn.Conv2d(self.out_chans, self.out_chans, kernel_size, stride, kernel_size//2)

#     def forward(self, x):
#         for mobivari in self.mobivari:
#             x = mobivari(x)
#         return self.final_conv(self.shuffle(self.conv(x)))

#     def flops(self, resolutions):
#         H,W = resolutions
#         flops = 0
#         for mobivari in self.mobivari: # self.mobivari
#             flops += mobivari.flops((H,W))
#         flops += H*W * self.kernel_size*self.kernel_size * self.dim * self.out_chans*(self.upscale**2) # self.conv
#         flops += H*W * self.kernel_size*self.kernel_size * self.out_chans * self.out_chans # self.final_conv
#         return flops

# class RAMiT_FFAT(nn.Module):
#     def __init__(self, in_chans=3, dim=64, depths=(1,6,4,4,6), num_heads=(4,4,4,4), head_dim=None, chsa_head_ratio=0.25,
#                  window_size=8, hidden_ratio=2.0, qkv_bias=True, mv_ver=1, exp_factor=1.2, expand_groups=4,
#                  act_layer=nn.GELU, norm_layer=ReshapeLayerNorm, tail_mv=2, weight_init='', target_mode='light_x2', img_norm=True,
#                  attn_drop=0.0, proj_drop=0.0, drop_path=0.0, helper=True, mv_act=nn.LeakyReLU,
#                  num_blocks = [1,4,6,6,8],
#                  heads = [1,2,4,8],
#                  ffn_expansion_factor = 2.66,
#                  bias = False,
#                  LayerNorm_type = 'WithBias',   ## Other option 'BiasFree'
#                  FFAT = False, **kwargs):
#         super(RAMiT_FFAT, self).__init__()
#         self.cat_dim = sum([dim//(4**x) for x in range(len(depths)-1)]) # added to use the torchinfor.summary
#         self.unit = 2**(len(depths)-2)*window_size

#         self.in_chans = in_chans
#         self.dim = dim
#         self.depths = depths
#         self.num_heads = num_heads
#         self.head_dim = head_dim
#         self.window_size = window_size
#         self.hidden_ratio = hidden_ratio
#         self.qkv_bias = qkv_bias
#         self.act_layer = act_layer
#         norm_layer = ReshapeLayerNorm if norm_layer == 'ReshapeLayerNorm' else norm_layer
#         self.norm_layer = norm_layer = ReshapeLayerNorm if norm_layer == 'ReshapeLayerNorm' else norm_layer
#         self.tail_mv = tail_mv

#         self.scale = int(target_mode[-1]) if target_mode in ['light_x2', 'light_x3', 'light_x4'] else 1
#         self.mean, self.std = mean_std(self.scale, target_mode)
#         self.target_mode = target_mode
#         self.img_norm = img_norm

#         dpr = [x.item() for x in torch.linspace(0, drop_path, sum(depths))]  # stochastic depth decay rule

#         self.shallow = ShallowModule(in_chans, dim, 3, 1)
#         self.stage1 = EncoderStage(depths[0], dim, num_heads[0], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
#                                    hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)


#         # FFAT
#         self.encoder_level1 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])

#         self.encoder_level2 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])

#         self.encoder_level3 = nn.Sequential(*[TransformerBlock(dim=dim, num_heads=heads[0], ffn_expansion_factor=ffn_expansion_factor, bias=bias, LayerNorm_type=LayerNorm_type) for i in range(num_blocks[0])])
#         # FFAT


#         self.down1 = Downsizing(dim, dim, norm_layer, mv_ver, mv_act)
#         self.stage2 = EncoderStage(depths[0], dim, num_heads[1], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
#                                    hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)
#         self.down2 = Downsizing(dim, dim, norm_layer, mv_ver, mv_act)
#         self.stage3 = EncoderStage(depths[0], dim, num_heads[2], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
#                                    hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)





#         if mv_ver==1:
#             self.mobivari_b = MobiVari1(self.cat_dim, 3, 1, act=mv_act, out_dim=dim)
#         elif mv_ver==2:
#             self.mobivari_b = MobiVari2(self.cat_dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
#             # NAFBlock---------------------------------------------
#         elif mv_ver==3:
#             self.mobivari_b = NAFBlock(c=self.dim, out_channel=dim)
#             # -------------------------------------------------------
#         self.act = act_layer()
#         self.norm = norm_layer(dim)
#         # self.bottleneck = Bottleneck(dim, len(depths)-1, act_layer, norm_layer, mv_ver, mv_act)

#         self.stage4 = EncoderStage(depths[0], dim, num_heads[3], chsa_head_ratio, FFAT, window_size, head_dim, qkv_bias, mv_ver,
#                                    hidden_ratio, act_layer, norm_layer, attn_drop, proj_drop, drop_path, helper, mv_act)







#         if mv_ver==1:
#             self.mobivari_mix = MobiVari1(dim+dim//4+dim//16+dim, 3, 1, act=mv_act, out_dim=dim)
#         elif mv_ver==2:
#             self.mobivari_mix = MobiVari2(dim+dim//4+dim//16+dim, 3, 1, act=mv_act, out_dim=dim, exp_factor=exp_factor, expand_groups=expand_groups)
#              # NAFBlock---------------------------------------------
#         elif mv_ver==3:
#             self.mobivari_mix = NAFBlock(c=dim+dim//4+dim//16+dim, out_channel=dim)
#             # -------------------------------------------------------
#         # self.attn_mix = HRAMi(dim, 3, 1, mv_ver, mv_act)


#         self.to_target = Reconstruction(target_mode, dim, 3, 1, tail_mv, mv_ver, mv_act, exp_factor, expand_groups)

#         self.apply(self._init_weights)

#     def forward_size_norm(self, x):
#         _,_,h,w = x.size()
#         padh = self.unit-(h%self.unit) if h%self.unit!=0 else 0
#         padw = self.unit-(w%self.unit) if w%self.unit!=0 else 0
#         x = TF.pad(x, (0,0,padw,padh))
#         return TF.normalize(x, self.mean, self.std) if self.img_norm else x

#     def forward(self, x):
#         _,_,H_ori,W_ori = x.size()
#         x = self.forward_size_norm(x)
#         B, C, H, W = x.size()
#         x_lq = x if self.scale == 1 else None # due to residual image (except for only SR)

#         shallow = self.shallow(x)
#         o1_, attn1 = self.stage1(shallow) # [B, C, H, W]


#         # FFAT
#         o1_ = self.encoder_level1(o1_)
#         # FFAT


#         o1 = self.down1(o1_) # [B, C, H//2, W//2]

#         o2_, attn2 = self.stage2(o1) # [B, C, H//2, W//2]


#         # FFAT
#         o2_ = self.encoder_level2(o2_)
#         # FFAT


#         o2 = self.down2(o2_) # [B, C, H//4, W//4]

#         o3_, attn3 = self.stage3(o2) # [B, C, H//4, W//4]


#         # FFAT
#         o3_ = self.encoder_level3(o3_)
#         # FFAT




# # Bottleneck
#         def shallow_down( x, r):
#           for _ in range(r):
#               x = F.max_pool2d(x, 2)
#           return F.leaky_relu(x)
#         x_list = [shallow, o1_, o2_, o3_]
#         xs = x_list[0]
#         new_x = []
#         for i in range(len(x_list[1:])):
#             x_ = x_list[i+1]+shallow_down(xs, i)
#             new_x.append(F.pixel_shuffle(x_, 2**i))
#         ob = self.norm(self.mobivari_b(torch.cat(new_x, dim=1)))
#         # ob = self.bottleneck([shallow, o1_, o2_, o3_]) # [B, C, H, W]



#         o4, attn4 = self.stage4(ob) # [B, C, H, W]






# # HRAMiT
#         attn_list = [attn1, attn2, attn3, attn4]
#         for i, attn in enumerate(attn_list[:-1]):
#             attn = F.pixel_shuffle(attn, 2**i)
#             x = attn if i==0 else torch.cat([x, attn], dim=1)
#         x = torch.cat([x, attn_list[-1]], dim=1)
#         mix = self.mobivari_mix(x)
#         # mix = self.attn_mix([attn1, attn2, attn3, attn4]) # [B, C, H, W]



#         o4 = o4 * mix # [B, C, H, W]

#         out = self.to_target(o4+shallow) # global skip connection
#         out = out + x_lq if self.scale == 1 else out # residual output (except for only SR)

#         out = denormalize(out, self.mean, self.std) if self.img_norm else out
#         out = out[:, :, :H_ori*self.scale, :W_ori*self.scale]
#         return out

#     def flops(self, resolutions):
#         H_ori,W_ori = resolutions
#         padh = self.unit-(H_ori%self.unit) if H_ori%self.unit!=0 else 0
#         padw = self.unit-(W_ori%self.unit) if W_ori%self.unit!=0 else 0
#         H,W = H_ori+padh,W_ori+padw
#         flops = 0
#         flops += self.shallow.flops((H,W))
#         flops += self.stage1.flops((H,W))
#         flops += self.down1.flops((H,W))
#         flops += self.stage2.flops((H//2,W//2))
#         flops += self.down2.flops((H//2,W//2))
#         flops += self.stage3.flops((H//4,W//4))
#         flops += self.bottleneck.flops((H,W))
#         flops += self.stage4.flops((H,W))
#         flops += self.attn_mix.flops((H,W))
#         flops += self.dim * H*W # o4 = o4*mix
#         flops += self.to_target.flops((H,W))
#         return flops

#     def _init_weights(self, m):
#         # Swin V2 manner
#         if isinstance(m, nn.Linear):
#             trunc_normal_(m.weight, std=.02)
#             if isinstance(m, nn.Linear) and m.bias is not None:
#                 nn.init.constant_(m.bias, 0)
#         elif isinstance(m, nn.LayerNorm):
#             nn.init.constant_(m.bias, 0)
#             nn.init.constant_(m.weight, 1.0)

#     @torch.jit.ignore
#     def no_weight_decay(self):
#         nwd = set()
#         for n, _ in self.named_parameters():
#             if 'relative_position_bias_table' in n:
#                 nwd.add(n)
#         return nwd






# # ----------------------------Shadow_R-------------------------
#     # @torch.no_grad()
#     # def _init_weights(module_list, scale=1, bias_fill=0, **kwargs):
#     #   """Initialize network weights.

#     #   Args:
#     #       module_list (list[nn.Module] | nn.Module): Modules to be initialized.
#     #       scale (float): Scale initialized weights, especially for residual
#     #           blocks. Default: 1.
#     #       bias_fill (float): The value to fill bias. Default: 0
#     #       kwargs (dict): Other arguments for initialization function.
#     #   """
#     #   if not isinstance(module_list, list):
#     #       module_list = [module_list]
#     #   for module in module_list:
#     #       for m in module.modules():
#     #           if isinstance(m, nn.Conv2d):
#     #               nn.init.kaiming_normal_(m.weight, **kwargs)
#     #               m.weight.data *= scale
#     #               if m.bias is not None:
#     #                   m.bias.data.fill_(bias_fill)
#     #           elif isinstance(m, nn.Linear):
#     #               nn.init.kaiming_normal_(m.weight, **kwargs)
#     #               m.weight.data *= scale
#     #               if m.bias is not None:
#     #                   m.bias.data.fill_(bias_fill)
#     #           elif isinstance(m, _BatchNorm):
#     #               nn.init.constant_(m.weight, 1)
#     #               if m.bias is not None:
#     #                   m.bias.data.fill_(bias_fill)

In [11]:
# model = RAMiT_FFAT(target_mode='light_realdn',img_norm=False, mv_ver=1)
# model.to('cuda')
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# # Create the model

# # Print the number of parameters
# print(f'The model has {count_parameters(model):,} trainable parameters')

In [12]:
# from torchinfo import summary
# summary(model, (1, 3, 256, 256))

<!-- ## Data loader -->

In [13]:
# from PIL import Image
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader
# import os
# import numpy as np
# import torch



# class ImageRestorationDataset(Dataset):
#     def __init__(self, root_dir, transform=None, num_samples=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.input_dir = os.path.join(root_dir, 'crop_inputs')
#         self.target_dir = os.path.join(root_dir, 'crop_targets')
#         self.input_filenames = [f for f in sorted(os.listdir(self.input_dir)) if os.path.isfile(os.path.join(self.input_dir, f))]
#         self.target_filenames = [f for f in sorted(os.listdir(self.target_dir)) if os.path.isfile(os.path.join(self.target_dir, f))]

#         if num_samples is not None:
#             self.input_filenames = self.input_filenames[:num_samples]
#             self.target_filenames = self.target_filenames[:num_samples]

#     def __len__(self):
#         return len(self.input_filenames)

#     def __getitem__(self, idx):
#         input_path = os.path.join(self.input_dir, self.input_filenames[idx])
#         target_path = os.path.join(self.target_dir, self.target_filenames[idx])
#         input_image = Image.open(input_path).convert('RGB')
#         target_image = Image.open(target_path).convert('RGB')


#         if self.transform:
#             input_image = self.transform(input_image)
#             target_image = self.transform(target_image)

#             # input_imaget = np.array(input_image, dtype=np.uint8)
#             # target_image = np.array(target_image, dtype=np.uint8)
#             # input_imaget = input_imaget.astype(np.float16)
#             # target_image = target_image.astype(np.float16)
#             # input_imaget = input_imaget.transpose(2, 1, 0)
#             # target_image = target_image.transpose(2, 1, 0)
#             # input_imaget = torch.from_numpy(input_imaget).type(torch.float16)
#             # target_image = torch.from_numpy(target_image).type(torch.float16)

#         # print(type(self.input_filenames))
#         return input_image, target_image

# # Define the transformations
# transform = transforms.Compose([
#     # transforms.Resize((128, 128)),  # Example resize
#     transforms.ToTensor(),  # Convert to PyTorch tensors
#     transforms.Normalize(mean=(0.0000, 0.0000, 0.0000), std=(1.0000, 1.0000, 1.0000))

# ])

# # Load the dataset
# train_dataset = ImageRestorationDataset(root_dir='/content/drive/MyDrive/istd_dataset/train', transform = transform, num_samples= 8400)
# val_dataset = ImageRestorationDataset(root_dir='/content/drive/MyDrive/istd_dataset/val',transform = transform, num_samples= 1680)
# # print(type(train_dataset))
# # Create DataLoaders
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# # Example: Iterate through the dataset
# # for inputs, targets in train_loader:
#     # print(f"Batch of input images has shape: {inputs.shape}")
#     # print(f"Batch of target images has shape: {targets.shape}")
#     # Further processing...


## DATA

In [14]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
import random
from PIL import Image

class ShadowRemovalDataset(Dataset):
    def __init__(self, gt_dir, lq_dir, transform=None, augment=False):
        self.gt_dir = gt_dir
        self.lq_dir = lq_dir
        self.transform = transform
        self.augment = augment

        # Filter out directories and non-image files
        self.gt_images = sorted([f for f in os.listdir(gt_dir) if os.path.isfile(os.path.join(gt_dir, f)) and f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))])
        self.lq_images = sorted([f for f in os.listdir(lq_dir) if os.path.isfile(os.path.join(lq_dir, f)) and f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))])
        # self.gt_images = self.gt_images[:80]
        # self.lq_images = self.gt_images[:80]
    def __len__(self):
        return len(self.gt_images)

    def augment_images(self, gt_image, lq_image):
        # Random rotation
        angle = random.choice([0, 90, 180, 270])
        gt_image = TF.rotate(gt_image, angle)
        lq_image = TF.rotate(lq_image, angle)

        # Random horizontal flipping
        if random.random() > 0.5:
            gt_image = TF.hflip(gt_image)
            lq_image = TF.hflip(lq_image)

        # Random vertical flipping
        if random.random() > 0.5:
            gt_image = TF.vflip(gt_image)
            lq_image = TF.vflip(lq_image)

        return gt_image, lq_image

    def random_crop(self, gt_image, lq_image):
        i, j, h, w = transforms.RandomCrop.get_params(
            #Random crop size
            gt_image, output_size=(360, 480))
        gt_image = TF.crop(gt_image, i, j, h, w)
        lq_image = TF.crop(lq_image, i, j, h, w)
        return gt_image, lq_image

    def __getitem__(self, idx):
        gt_img_path = os.path.join(self.gt_dir, self.gt_images[idx])
        lq_img_path = os.path.join(self.lq_dir, self.lq_images[idx])

        gt_image = Image.open(gt_img_path).convert("RGB")
        lq_image = Image.open(lq_img_path).convert("RGB")

        if self.augment:
            #gt_image, lq_image = self.augment_images(gt_image, lq_image)
            gt_image, lq_image = self.random_crop(gt_image, lq_image)

        if self.transform:
            gt_image = self.transform(gt_image)
            lq_image = self.transform(lq_image)

        return lq_image, gt_image

# Load the dataset
transform = transforms.Compose([
    transforms.ToTensor()
])

# Define paths to your dataset in Google Drive
train_gt_dir = r'C:\Users\Admin\Desktop\shadow_removal_results\ISTD\train\targets+'
train_lq_dir = r'C:\Users\Admin\Desktop\shadow_removal_results\ISTD\train\inputs'
val_gt_dir = r'C:\Users\Admin\Desktop\shadow_removal_results\ISTD\val\targets+'
val_lq_dir = r'C:\Users\Admin\Desktop\shadow_removal_results\ISTD\val\inputs'

# Replace with your dataset path
train_dataset = ShadowRemovalDataset(gt_dir=train_gt_dir, lq_dir=train_lq_dir, transform=transform, augment=True)
val_dataset = ShadowRemovalDataset(gt_dir=val_gt_dir, lq_dir=val_lq_dir, transform=transform, augment=False)

batch_size = 2 # Set the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, pin_memory=True)

# Check the number of files (samples) in the datasets
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")

# Check the number of batches in the DataLoaders
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Admin\\Desktop\\shadow_removal_results\\ISTD\\train\\targets+'

## Training

In [ ]:
# import time
# from tqdm import tqdm
# from torchmetrics.functional import structural_similarity_index_measure as ssim
# from lpips import LPIPS

# # transfrom the data from tensors to numpys to calculate the PSNR
# def tensor_to_numpy(tensor):
#     tensor = tensor.permute(1, 2, 0).cpu().numpy()  # Change from (C, H, W) to (H, W, C)
#     tensor = (tensor * 255).astype(np.uint8)  # Optional: Scale to [0, 255] and convert to uint8
#     return tensor
# # ----------------------------------------------------

# # uste the transfromed data to calculate the PSNR
# def calculate_psnr(img1_tensor, img2_tensor):
#     img1 = tensor_to_numpy(img1_tensor)
#     img2 = tensor_to_numpy(img2_tensor)
#     psnr_value = psnr(img1, img2)
#     return psnr_value
# # -------------------------------------------------

# lpips_loss = LPIPS(net='alex').to('cuda')

In [ ]:
# def evaluate(model: nn.Module, val_loader) -> float:
#     model.eval()  # Turn on evaluation mode

#     # initialize all losses
#     criterion = nn.L1Loss()
#     # lpips_loss = LPIPS(net='alex').to('cuda')
#     # -----------------------------------------------

#     # initialize losses values
#     total_loss = 0.0
#     total_psnr = 0.0
#     total_ssim = 0.0
#     total_lpips = 0.0
#   # -----------------------------------------

# # list of low result
#     output_l = []
#     targets_l = []
#     inputs_l = []
# # ------------------------------------

#     # list of top results
#     tev_out = []
#     tev_inputs = []
#     tev_targets = []
# # ---------------------------------------

#     with torch.no_grad():
#         # with tqdm(total=len(val_loader), desc="Validating", unit="batch") as val_pbar:
#             for batch_idx, (inputs, targets) in enumerate(val_loader):
#                 inputs, targets = inputs.to('cuda'), targets.to('cuda')
#                 output = model(inputs)

#                 # L1 loss
#                 loss = criterion(output, targets)
#                 # ------------------------------------------

#                 # ssim loss
#                 ssim_L = ssim(output, targets, data_range=1.0)
#                 # -------------------------------------------

#                 # lpips loss
#                 # batch_lpips = 0.0
#                 img1 = (targets * 2) - 1
#                 img2 = (output * 2) - 1
#                 # for i in range(inputs.size(0)):
#                 #   batch_lpips += lpips_loss(img1[i], img2[i]).item()
#                 # batch_lpips /= inputs.size(0)
#                 batch_lpips = lpips_loss(img1, img2).mean()
#                 # -----------------------------------------------

#                 # Calculate the PSNR for each batch
#                 batch_psnr = 0
#                 for i in range(inputs.size(0)):
#                     batch_psnr += calculate_psnr(output[i].detach(), targets[i])
#                 batch_psnr /= inputs.size(0)
#                 # ------------------------------------------------------



#                 # total loss for all losses
#                 total_loss += loss.item() # total loss for L1
#                 total_psnr += batch_psnr # total loss for psnr
#                 total_ssim += ssim_L.item() # total loss for ssim
#                 total_lpips += batch_lpips # total loss for lpips
#                 # -----------------------------------------------------------



#                 # top and low results
#                 if batch_psnr > 10 and batch_psnr <= 16:
#                   output_l.append(output.to('cpu'))
#                   targets_l.append(targets.to('cpu'))
#                   inputs_l.append(inputs.to('cpu'))
#                 elif batch_psnr >= 29:
#                   tev_out.append(output.to('cpu'))
#                   tev_inputs.append(inputs.to('cpu'))
#                   tev_targets.append(targets.to('cpu'))
#                   # ------------------------------------------


#                 # Update progress bar for the current batch
#                 # val_pbar.set_postfix({"loss": loss.item(), "batch_psnr": batch_psnr, 'batch_ssim': ssim_L.item(), 'batch_lpips': batch_lpips})
#                 # val_pbar.update(1)

#     avg_loss = total_loss / len(val_loader)
#     avg_psnr = total_psnr / len(val_loader)
#     avg_ssim = total_ssim / len(val_loader)
#     avg_lpips = total_lpips / len(val_loader)
#     return  tev_out, tev_inputs, tev_targets, output_l, targets_l, inputs_l, avg_loss,  avg_psnr, avg_ssim, avg_lpips

In [ ]:
# def train(model: nn.Module, train_loader, epochs) -> None:
#     model.train()  # Turn on train mode

#     lr = 0.001
#     criterion = nn.L1Loss()
#     # lpips_loss = LPIPS(net='alex').to('cuda')

#     optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.99), eps=1e-08)

#     top_output = []
#     top_inputs = []
#     top_targets = []
#     low_output = []
#     low_inputs = []
#     low_targets = []
#     for epoch in range(1, epochs+1):
#             total_loss = 0.0
#             total_psnr = 0.0
#             total_ssim = 0.0
#             total_lpips = 0.0
#             start_time = time.time()
#             batch_num = 1
#         # Initialize tqdm progress bar
#         # with tqdm(total=len(train_loader), desc=f"Epoch {epoch}/{epochs}", unit="batch") as pbar:
#             for batch_idx, (inputs, targets) in enumerate(train_loader):
#                 inputs, targets = inputs.to('cuda'), targets.to('cuda')
#                 optimizer.zero_grad()
#                 output = model(inputs)

#                 # L1 loss + backprop
#                 loss = criterion(output, targets)
#                 loss.backward()
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
#                 optimizer.step()
#                 # ------------------------------------------------

#                 # ssim loss
#                 ssim_L = ssim(output, targets, data_range=1.0)
#                 # -----------------------------------------------

#                 # lpips loss
#                 batch_lpips = 0.0
#                 img1 = (targets * 2) - 1
#                 img2 = (output * 2) - 1
#                 # for i in range(inputs.size(0)):
#                 #   batch_lpips += lpips_loss(img1[i], img2[i]).item()
#                 # batch_lpips /= inputs.size(0)
#                 with torch.no_grad():
#                   batch_lpips = lpips_loss(img1, img2).mean()
#                 # -----------------------------------------------




#                 # Calculate the PSNR for each batch
#                 batch_psnr = 0
#                 for i in range(inputs.size(0)):
#                     batch_psnr += calculate_psnr(output[i].detach(), targets[i])
#                 batch_psnr /= inputs.size(0)

#                 # ------------------------------------------------------

#                 # total loss for all losses
#                 total_loss += loss.item() # total loss for L1
#                 total_psnr += batch_psnr # total loss for psnr
#                 total_ssim += ssim_L.item() # total loss for ssim
#                 total_lpips += batch_lpips # total loss for lpips
#                 # -----------------------------------------------------

#                 print(f' batch_num: {batch_num} | loss: {loss.item():.5f} | psnr_btch: {batch_psnr:.5f} | ssim: {ssim_L.item():.5f}| lpips: {batch_lpips:.5f}')
#                 batch_num += 1
#                 # Update the best enhanced images
#                 if batch_psnr > 10 and batch_psnr < 16:
#                     low_output.append(output.to('cpu'))
#                     low_inputs.append(inputs.to('cpu'))
#                     low_targets.append(targets.to('cpu'))
#                 elif batch_psnr > 29:
#                     top_output.append(output.to('cpu'))
#                     top_inputs.append(inputs.to('cpu'))
#                     top_targets.append(targets.to('cpu'))
#                 # -----------------------------------------------------------

#                 # Update progress bar
#                 # pbar.set_postfix({"loss": loss.item(), "batch_psnr": batch_psnr})
#                 # pbar.update(1)

#             # Log end of epoch details
#             cur_loss = total_loss / len(train_loader)
#             cur_psnr = total_psnr / len(train_loader)
#             cur_ssim = total_ssim / len(train_loader)
#             cur_lpips = total_lpips / len(train_loader)
#             # val_loss, e_out, total_psnr = evaluate(model, val_loader)
#             print(f'End of Epoch {epoch}  | LOSS {cur_loss:5.4f} | PSNR {cur_psnr:.4f} | SSIM: {ssim_L.item():.5f} | LPIPS: {batch_lpips:.5f}')
#             tev_out, tev_inputs, tev_targets, output_l, targets_l, inputs_l, avg_loss,  avg_psnr, avg_ssim, avg_lpips = evaluate(model, val_loader)
#             print(f'\n End of valadation {epoch} | LOSS: {avg_loss:5.4f} | PSNR: {avg_psnr:.4f} | SSIM: {avg_ssim:.5f} | LPIPS: {avg_lpips:.5f}')

#     print('Training completed.')
#     return top_output, top_inputs, top_targets, low_output, low_inputs, low_targets, tev_out, tev_inputs, tev_targets, output_l, targets_l, inputs_l

## MD_Trainer

In [ ]:
# import os
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# from torchvision.utils import save_image
# from tqdm import tqdm
# from skimage.metrics import structural_similarity as ssim
# from skimage.metrics import peak_signal_noise_ratio as psnr
# from lpips import LPIPS
# import torchvision.models as models

# # Constants
# LEARNING_RATE = 0.001
# NUM_EPOCHS = 100
# SAVE_INTERVAL = 5
# START_EPOCH = 0  # Start from epoch 80
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# #G1.load_state_dict(torch.load(r"C:\Users\nasef\Desktop\SBSR\Models\SBSR32\SBSR30.pth"))


# optimizer_G = optim.Adam(G1.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))

# # Add the cosine annealing scheduler
# scheduler_G = optim.lr_scheduler.CosineAnnealingLR(optimizer_G, T_max=NUM_EPOCHS - START_EPOCH, eta_min=0.000001)

# # Loss functions
# l1_loss = nn.L1Loss()
# mse_loss = nn.MSELoss()
# lpips_loss_fn = LPIPS(net='alex').to(DEVICE)  # Using AlexNet for LPIPS loss

# # Define the VGG loss
# class VGGLoss(nn.Module):
#     def __init__(self, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
#         super(VGGLoss, self).__init__()
#         vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features.to(device).eval()  # Move VGG layers to device and set to eval mode
#         self.vgg_layers = nn.Sequential(*list(vgg)[:36]).to(device)
#         self.device = device
#         for param in self.vgg_layers.parameters():
#             param.requires_grad = False

#     def forward(self, x, y):
#         x = x.to(self.device)
#         y = y.to(self.device)
#         x_vgg = self.vgg_layers(x)
#         y_vgg = self.vgg_layers(y)
#         loss = nn.functional.l1_loss(x_vgg, y_vgg)
#         return loss

# # Define Focal Frequency Loss
# class FocalFrequencyLoss(nn.Module):
#     def __init__(self, alpha=1.0, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
#         super(FocalFrequencyLoss, self).__init__()
#         self.alpha = alpha
#         self.device = device

#     def forward(self, input, target):
#         input = input.to(self.device)
#         target = target.to(self.device)
#         input_fft = torch.fft.fft2(input)
#         target_fft = torch.fft.fft2(target)
#         diff = input_fft - target_fft
#         abs_diff = torch.abs(diff)
#         loss = torch.pow(abs_diff, self.alpha)
#         return torch.mean(loss)

# # Loss weights
# lambda_vgg = 0.01
# lambda_ff = 0.1

# # Combined loss
# class CombinedLoss(nn.Module):
#     def __init__(self, lambda_vgg, lambda_ff, device=DEVICE):
#         super(CombinedLoss, self).__init__()
#         self.lambda_vgg = lambda_vgg
#         self.lambda_ff = lambda_ff
#         self.vgg_loss = VGGLoss(device=device)
#         self.ff_loss = FocalFrequencyLoss(device=device)

#     def forward(self, input, target):
#         l1 = l1_loss(input, target)
#         vgg = self.vgg_loss(input, target)
#         ff = self.ff_loss(input, target)
#         return l1 + self.lambda_vgg * vgg + self.lambda_ff * ff

# # Example usage
# combined_loss = CombinedLoss(lambda_vgg=lambda_vgg, lambda_ff=lambda_ff, device=DEVICE)

# # Directories for checkpoints and samples
# checkpoint_dir = r'C:\Users\Admin\Desktop\AI_RAMiT\RAMiT'
# sample_dir = r'C:\Users\Admin\Desktop\AI_RAMiT\RAMiT'
# os.makedirs(checkpoint_dir, exist_ok=True)
# os.makedirs(sample_dir, exist_ok=True)
# save_interval = SAVE_INTERVAL  # Save checkpoint and samples every 5 epochs

# for epoch in range(START_EPOCH, NUM_EPOCHS):
#     # Training
#     G1.train()
#     total_g_loss = 0.0
#     total_l1_loss = 0.0
#     with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{NUM_EPOCHS}', unit='batch') as pbar:
#         for i, (real_shadow, real_free) in enumerate(train_loader):
#             real_shadow = real_shadow.to(DEVICE)
#             real_free = real_free.to(DEVICE)
#             optimizer_G.zero_grad()

#             pred_free = G1(real_shadow)
#             # Generator loss
#             g1_loss = l1_loss(pred_free, real_free)
#             g_loss = g1_loss
#             g_loss.backward()
#             torch.nn.utils.clip_grad_norm_(G1.parameters(), max_norm=1.0)
#             optimizer_G.step()

#             # Accumulate losses
#             total_g_loss += g_loss.item()
#             total_l1_loss += g1_loss.item()

#             pbar.set_postfix(G_loss=g_loss.item(), G1_loss=g1_loss.item())
#             pbar.update(1)

#     # Calculate average losses
#     avg_g_loss = total_g_loss / len(train_loader)
#     avg_l1_loss = total_l1_loss / len(train_loader)

#     print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}] - Average G Loss: {avg_g_loss:.4f}")
#     scheduler_G.step()

#     m = 0
#     # Validation and Checkpoints
#     if (epoch + 1+70) % save_interval == 0:
#         torch.cuda.empty_cache()
#         G1.eval()
#         with torch.no_grad():
#             val_g_loss = 0.0
#             val_l1_loss = 0.0
#             val_ssim_loss = 0.0
#             val_lpips_loss = 0.0
#             val_psnr_mask = 0.0

#             with tqdm(total=len(val_loader), desc=f'Validation Epoch {epoch + 1}/{NUM_EPOCHS}', unit='batch') as pbar_val:
#                 for i, (real_shadow, real_free) in enumerate(val_loader):
#                     real_shadow = real_shadow.to(DEVICE)
#                     real_free = real_free.to(DEVICE)

#                     pred_free = G1(real_shadow)
#                     # Generator loss
#                     g1_loss = l1_loss(pred_free, real_free)
#                     g_loss = g1_loss

#                     val_g_loss += g_loss.item()
#                     val_l1_loss += g1_loss.item()

#                     # Calculate SSIM and LPIPS
#                     pred_free_np = pred_free.cpu().numpy().transpose(0, 2, 3, 1)
#                     real_free_np = real_free.cpu().numpy().transpose(0, 2, 3, 1)

#                     #ssim_val = ssim(pred_free_np, real_free_np, data_range=1, channel_axis=-1)
#                     lpips_val = lpips_loss_fn(pred_free, real_free).mean().item()  # Average LPIPS over batch
#                     #val_ssim_loss += ssim_val
#                     val_lpips_loss += lpips_val

#                     # Calculate PSNR for final shadow-free images
#                     for j in range(real_free_np.shape[0]):
#                         val_psnr_mask += psnr(real_free_np[j], pred_free_np[j], data_range=1)
#                         val_ssim_loss += ssim(real_free_np[j], pred_free_np[j], data_range=1, channel_axis=-1)


#                     pbar_val.update(1)
#                     save_image(pred_free, os.path.join(sample_dir, f'pred_free_epoch_{epoch + 1}_{m}.png'))
#                     m += 1

#             # Averaging over the entire validation set
#             val_g_loss /= len(val_loader)
#             val_l1_loss /= len(val_loader)
#             val_ssim_loss /= len(val_loader.dataset)
#             val_lpips_loss /= len(val_loader)
#             val_psnr_mask /= len(val_loader.dataset)

#             print(f"Validation after Epoch [{epoch + 1}/{NUM_EPOCHS}] "
#                   f"Val G Loss: {val_g_loss:.4f}, Val L1 Loss: {val_l1_loss:.4f}, "
#                   f"PSNR_MASK: {val_psnr_mask:.4f}, SSIM: {val_ssim_loss:.4f}, LPIPS: {val_lpips_loss:.4f}")

#             # Save model checkpoints
#             torch.save(G1.state_dict(), os.path.join(checkpoint_dir, f'SBSR{epoch + 1}.pth'))

# print("Training completed")


## RMAiT

In [ ]:
# import os
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# from torchvision.utils import save_image
# from tqdm import tqdm
# from skimage.metrics import structural_similarity as ssim
# from skimage.metrics import peak_signal_noise_ratio as psnr
# from lpips import LPIPS
# import torchvision.models as models

# # Constants
# LEARNING_RATE = 0.001
# NUM_EPOCHS = 500
# SAVE_INTERVAL = 5
# START_EPOCH = 0  # Start from epoch 80
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# #G1.load_state_dict(torch.load(r"C:\Users\nasef\Desktop\SBSR\Models\SBSR32\SBSR30.pth"))


# optimizer_G = optim.Adam(G1.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))

# # Add the cosine annealing scheduler
# scheduler_G = optim.lr_scheduler.CosineAnnealingLR(optimizer_G, T_max=NUM_EPOCHS - START_EPOCH, eta_min=0.000001)

# # Loss functions
# l1_loss = nn.L1Loss()
# mse_loss = nn.MSELoss()
# lpips_loss_fn = LPIPS(net='alex').to(DEVICE)  # Using AlexNet for LPIPS loss

# # Define the VGG loss
# class VGGLoss(nn.Module):
#     def __init__(self, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
#         super(VGGLoss, self).__init__()
#         vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features.to(device).eval()  # Move VGG layers to device and set to eval mode
#         self.vgg_layers = nn.Sequential(*list(vgg)[:36]).to(device)
#         self.device = device
#         for param in self.vgg_layers.parameters():
#             param.requires_grad = False

#     def forward(self, x, y):
#         x = x.to(self.device)
#         y = y.to(self.device)
#         x_vgg = self.vgg_layers(x)
#         y_vgg = self.vgg_layers(y)
#         loss = nn.functional.l1_loss(x_vgg, y_vgg)
#         return loss

# # Define Focal Frequency Loss
# class FocalFrequencyLoss(nn.Module):
#     def __init__(self, alpha=1.0, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
#         super(FocalFrequencyLoss, self).__init__()
#         self.alpha = alpha
#         self.device = device

#     def forward(self, input, target):
#         input = input.to(self.device)
#         target = target.to(self.device)
#         input_fft = torch.fft.fft2(input)
#         target_fft = torch.fft.fft2(target)
#         diff = input_fft - target_fft
#         abs_diff = torch.abs(diff)
#         loss = torch.pow(abs_diff, self.alpha)
#         return torch.mean(loss)

# # Loss weights
# lambda_vgg = 0.01
# lambda_ff = 0.1

# # Combined loss
# class CombinedLoss(nn.Module):
#     def __init__(self, lambda_vgg, lambda_ff, device=DEVICE):
#         super(CombinedLoss, self).__init__()
#         self.lambda_vgg = lambda_vgg
#         self.lambda_ff = lambda_ff
#         self.vgg_loss = VGGLoss(device=device)
#         self.ff_loss = FocalFrequencyLoss(device=device)

#     def forward(self, input, target):
#         l1 = l1_loss(input, target)
#         vgg = self.vgg_loss(input, target)
#         ff = self.ff_loss(input, target)
#         return l1 + self.lambda_vgg * vgg + self.lambda_ff * ff

# # Example usage
# combined_loss = CombinedLoss(lambda_vgg=lambda_vgg, lambda_ff=lambda_ff, device=DEVICE)

# # Directories for checkpoints and samples
# checkpoint_dir = r'C:\Users\Admin\Desktop\AI_RAMiT\RAMiT'
# sample_dir = r'C:\Users\Admin\Desktop\AI_RAMiT\RAMiT'
# os.makedirs(checkpoint_dir, exist_ok=True)
# os.makedirs(sample_dir, exist_ok=True)
# save_interval = SAVE_INTERVAL  # Save checkpoint and samples every 5 epochs

# for epoch in range(START_EPOCH, NUM_EPOCHS):
#     # Training
#     G1.train()
#     total_g_loss = 0.0
#     total_l1_loss = 0.0
#     with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{NUM_EPOCHS}', unit='batch') as pbar:
#         for i, (real_shadow, real_free) in enumerate(train_loader):
#             real_shadow = real_shadow.to(DEVICE)
#             real_free = real_free.to(DEVICE)
#             optimizer_G.zero_grad()

#             pred_free = G1(real_shadow)
#             # Generator loss
#             g1_loss = l1_loss(pred_free, real_free)
#             g_loss = g1_loss
#             g_loss.backward()
#             torch.nn.utils.clip_grad_norm_(G1.parameters(), max_norm=1.0)
#             optimizer_G.step()

#             # Accumulate losses
#             total_g_loss += g_loss.item()
#             total_l1_loss += g1_loss.item()

#             pbar.set_postfix(G_loss=g_loss.item(), G1_loss=g1_loss.item())
#             pbar.update(1)

#     # Calculate average losses
#     avg_g_loss = total_g_loss / len(train_loader)
#     avg_l1_loss = total_l1_loss / len(train_loader)

#     print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}] - Average G Loss: {avg_g_loss:.4f}")
#     scheduler_G.step()

#     m = 0
#     # Validation and Checkpoints
#     if (epoch + 1+70) % save_interval == 0:
#         torch.cuda.empty_cache()
#         G1.eval()
#         with torch.no_grad():
#             val_g_loss = 0.0
#             val_l1_loss = 0.0
#             val_ssim_loss = 0.0
#             val_lpips_loss = 0.0
#             val_psnr_mask = 0.0

#             with tqdm(total=len(val_loader), desc=f'Validation Epoch {epoch + 1}/{NUM_EPOCHS}', unit='batch') as pbar_val:
#                 for i, (real_shadow, real_free) in enumerate(val_loader):
#                     real_shadow = real_shadow.to(DEVICE)
#                     real_free = real_free.to(DEVICE)

#                     pred_free = G1(real_shadow)
#                     # Generator loss
#                     g1_loss = l1_loss(pred_free, real_free)
#                     g_loss = g1_loss

#                     val_g_loss += g_loss.item()
#                     val_l1_loss += g1_loss.item()

#                     # Calculate SSIM and LPIPS
#                     pred_free_np = pred_free.cpu().numpy().transpose(0, 2, 3, 1)
#                     real_free_np = real_free.cpu().numpy().transpose(0, 2, 3, 1)

#                     #ssim_val = ssim(pred_free_np, real_free_np, data_range=1, channel_axis=-1)
#                     lpips_val = lpips_loss_fn(pred_free, real_free).mean().item()  # Average LPIPS over batch
#                     #val_ssim_loss += ssim_val
#                     val_lpips_loss += lpips_val

#                     # Calculate PSNR for final shadow-free images
#                     for j in range(real_free_np.shape[0]):
#                         val_psnr_mask += psnr(real_free_np[j], pred_free_np[j], data_range=1)
#                         val_ssim_loss += ssim(real_free_np[j], pred_free_np[j], data_range=1, channel_axis=-1)


#                     pbar_val.update(1)
#                     save_image(pred_free, os.path.join(sample_dir, f'pred_free_epoch_{epoch + 1}_{m}.png'))
#                     m += 1

#             # Averaging over the entire validation set
#             val_g_loss /= len(val_loader)
#             val_l1_loss /= len(val_loader)
#             val_ssim_loss /= len(val_loader.dataset)
#             val_lpips_loss /= len(val_loader)
#             val_psnr_mask /= len(val_loader.dataset)

#             print(f"Validation after Epoch [{epoch + 1}/{NUM_EPOCHS}] "
#                   f"Val G Loss: {val_g_loss:.4f}, Val L1 Loss: {val_l1_loss:.4f}, "
#                   f"PSNR_MASK: {val_psnr_mask:.4f}, SSIM: {val_ssim_loss:.4f}, LPIPS: {val_lpips_loss:.4f}")

#             # Save model checkpoints
#             torch.save(G1.state_dict(), os.path.join(checkpoint_dir, f'SBSR{epoch + 1}.pth'))

# print("Training completed")


## FFAT_RAMiT

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from lpips import LPIPS
import torchvision.models as models

# Constants
LEARNING_RATE = 0.001
NUM_EPOCHS = 500
SAVE_INTERVAL = 5
START_EPOCH = 0  # Start from epoch 80
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#G1.load_state_dict(torch.load(r"C:\Users\nasef\Desktop\SBSR\Models\SBSR32\SBSR30.pth"))


optimizer_G = optim.Adam(G1.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))

# Add the cosine annealing scheduler
scheduler_G = optim.lr_scheduler.CosineAnnealingLR(optimizer_G, T_max=NUM_EPOCHS - START_EPOCH, eta_min=0.000001)

# Loss functions
l1_loss = nn.L1Loss()
mse_loss = nn.MSELoss()
lpips_loss_fn = LPIPS(net='alex').to(DEVICE)  # Using AlexNet for LPIPS loss

# Define the VGG loss
class VGGLoss(nn.Module):
    def __init__(self, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super(VGGLoss, self).__init__()
        vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features.to(device).eval()  # Move VGG layers to device and set to eval mode
        self.vgg_layers = nn.Sequential(*list(vgg)[:36]).to(device)
        self.device = device
        for param in self.vgg_layers.parameters():
            param.requires_grad = False

    def forward(self, x, y):
        x = x.to(self.device)
        y = y.to(self.device)
        x_vgg = self.vgg_layers(x)
        y_vgg = self.vgg_layers(y)
        loss = nn.functional.l1_loss(x_vgg, y_vgg)
        return loss

# Define Focal Frequency Loss
class FocalFrequencyLoss(nn.Module):
    def __init__(self, alpha=1.0, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super(FocalFrequencyLoss, self).__init__()
        self.alpha = alpha
        self.device = device

    def forward(self, input, target):
        input = input.to(self.device)
        target = target.to(self.device)
        input_fft = torch.fft.fft2(input)
        target_fft = torch.fft.fft2(target)
        diff = input_fft - target_fft
        abs_diff = torch.abs(diff)
        loss = torch.pow(abs_diff, self.alpha)
        return torch.mean(loss)

# Loss weights
lambda_vgg = 0.01
lambda_ff = 0.1

# Combined loss
class CombinedLoss(nn.Module):
    def __init__(self, lambda_vgg, lambda_ff, device=DEVICE):
        super(CombinedLoss, self).__init__()
        self.lambda_vgg = lambda_vgg
        self.lambda_ff = lambda_ff
        self.vgg_loss = VGGLoss(device=device)
        self.ff_loss = FocalFrequencyLoss(device=device)

    def forward(self, input, target):
        l1 = l1_loss(input, target)
        vgg = self.vgg_loss(input, target)
        ff = self.ff_loss(input, target)
        return l1 + self.lambda_vgg * vgg + self.lambda_ff * ff

# Example usage
combined_loss = CombinedLoss(lambda_vgg=lambda_vgg, lambda_ff=lambda_ff, device=DEVICE)

# Directories for checkpoints and samples
checkpoint_dir = r'C:\Users\Admin\Desktop\AI_RAMiT\FFAT_RAMiT'
sample_dir = r'C:\Users\Admin\Desktop\AI_RAMiT\FFAT_RAMiT_s'
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(sample_dir, exist_ok=True)
save_interval = SAVE_INTERVAL  # Save checkpoint and samples every 5 epochs

for epoch in range(START_EPOCH, NUM_EPOCHS):
    # Training
    G1.train()
    total_g_loss = 0.0
    total_l1_loss = 0.0
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{NUM_EPOCHS}', unit='batch') as pbar:
        for i, (real_shadow, real_free) in enumerate(train_loader):
            real_shadow = real_shadow.to(DEVICE)
            real_free = real_free.to(DEVICE)
            optimizer_G.zero_grad()

            pred_free = G1(real_shadow)
            # Generator loss
            g1_loss = l1_loss(pred_free, real_free)
            g_loss = g1_loss
            g_loss.backward()
            torch.nn.utils.clip_grad_norm_(G1.parameters(), max_norm=1.0)
            optimizer_G.step()

            # Accumulate losses
            total_g_loss += g_loss.item()
            total_l1_loss += g1_loss.item()

            pbar.set_postfix(G_loss=g_loss.item(), G1_loss=g1_loss.item())
            pbar.update(1)

    # Calculate average losses
    avg_g_loss = total_g_loss / len(train_loader)
    avg_l1_loss = total_l1_loss / len(train_loader)

    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}] - Average G Loss: {avg_g_loss:.4f}")
    scheduler_G.step()

    m = 0
    # Validation and Checkpoints
    if (epoch + 1) % save_interval == 0:
        torch.cuda.empty_cache()
        G1.eval()
        with torch.no_grad():
            val_g_loss = 0.0
            val_l1_loss = 0.0
            val_ssim_loss = 0.0
            val_lpips_loss = 0.0
            val_psnr_mask = 0.0

            with tqdm(total=len(val_loader), desc=f'Validation Epoch {epoch + 1}/{NUM_EPOCHS}', unit='batch') as pbar_val:
                for i, (real_shadow, real_free) in enumerate(val_loader):
                    real_shadow = real_shadow.to(DEVICE)
                    real_free = real_free.to(DEVICE)

                    pred_free = G1(real_shadow)
                    # Generator loss
                    g1_loss = l1_loss(pred_free, real_free)
                    g_loss = g1_loss

                    val_g_loss += g_loss.item()
                    val_l1_loss += g1_loss.item()

                    # Calculate SSIM and LPIPS
                    pred_free_np = pred_free.cpu().numpy().transpose(0, 2, 3, 1)
                    real_free_np = real_free.cpu().numpy().transpose(0, 2, 3, 1)

                    #ssim_val = ssim(pred_free_np, real_free_np, data_range=1, channel_axis=-1)
                    lpips_val = lpips_loss_fn(pred_free, real_free).mean().item()  # Average LPIPS over batch
                    #val_ssim_loss += ssim_val
                    val_lpips_loss += lpips_val

                    # Calculate PSNR for final shadow-free images
                    for j in range(real_free_np.shape[0]):
                        val_psnr_mask += psnr(real_free_np[j], pred_free_np[j], data_range=1)
                        val_ssim_loss += ssim(real_free_np[j], pred_free_np[j], data_range=1, channel_axis=-1)


                    pbar_val.update(1)
                    save_image(pred_free, os.path.join(sample_dir, f'pred_free_epoch_{epoch + 1}_{m}.png'))
                    m += 1

            # Averaging over the entire validation set
            val_g_loss /= len(val_loader)
            val_l1_loss /= len(val_loader)
            val_ssim_loss /= len(val_loader.dataset)
            val_lpips_loss /= len(val_loader)
            val_psnr_mask /= len(val_loader.dataset)

            print(f"Validation after Epoch [{epoch + 1}/{NUM_EPOCHS}] "
                  f"Val G Loss: {val_g_loss:.4f}, Val L1 Loss: {val_l1_loss:.4f}, "
                  f"PSNR_MASK: {val_psnr_mask:.4f}, SSIM: {val_ssim_loss:.4f}, LPIPS: {val_lpips_loss:.4f}")

            # Save model checkpoints
            torch.save(G1.state_dict(), os.path.join(checkpoint_dir, f'SBSR{epoch + 1}.pth'))

print("Training completed")


## RAMiT eval

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from lpips import LPIPS
import torchvision.models as models
e = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G1 = RAMiT_FFAT(target_mode='light_realdn',img_norm=False, mv_ver = 3)
while e<=500:
    from collections import OrderedDict

    # Load checkpoint and create new state_dict with 'module.' prefix if needed
    checkpoint = torch.load(f'C:/Users/Admin/Desktop/AI_RAMiT/FFAT_RAMiT/SBSR{e}.pth', map_location='cuda')
    state_dict = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint
    new_state_dict = OrderedDict()

    # Add prefix 'module.' if needed
    for k, v in state_dict.items():
        name = k if k.startswith('module.') else 'module.' + k
        new_state_dict[name] = v


    # Apply DataParallel and move to GPU
    G1 = G1.to('cuda')
    G1 = torch.nn.DataParallel(G1, device_ids=[0, 1])

    # G1.to('cuda')
    epoch = 1
    NUM_EPOCHS = 1
# optimizer_G = optim.Adam(G1.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))

# # # Add the cosine annealing scheduler
# scheduler_G = optim.lr_scheduler.CosineAnnealingLR(optimizer_G, T_max=NUM_EPOCHS - START_EPOCH, eta_min=0.000001)

# Loss functions
    l1_loss = nn.L1Loss()
    mse_loss = nn.MSELoss()
    lpips_loss_fn = LPIPS(net='alex').to(DEVICE)  # Using AlexNet for LPIPS loss
    with torch.no_grad():
                val_g_loss = 0.0
                val_l1_loss = 0.0
                val_ssim_loss = 0.0
                val_ssim_crop4_loss = 0.0
                val_lpips_loss = 0.0
                val_psnr_mask = 0.0
                val_psnr_y = 0.0

                with tqdm(total=len(val_loader), desc=f'Validation Epoch {epoch + 1}/{NUM_EPOCHS}', unit='batch') as pbar_val:
                    for i, (real_shadow, real_free) in enumerate(val_loader):
                        real_shadow = real_shadow.to(DEVICE)
                        real_free = real_free.to(DEVICE)

                        pred_free = G1(real_shadow)
                        # Generator loss
                        g1_loss = l1_loss(pred_free, real_free)
                        # g_loss = combined_loss(pred_free, real_free)

                        # val_g_loss += g_loss.item()
                        val_l1_loss += g1_loss.item()

                        # Calculate SSIM, LPIPS, PSNR, PSNR Y, and SSIM crop 4
                        pred_free_np = pred_free.cpu().numpy().transpose(0, 2, 3, 1)
                        real_free_np = real_free.cpu().numpy().transpose(0, 2, 3, 1)

                        lpips_val = lpips_loss_fn(pred_free, real_free).mean().item()  # Average LPIPS over batch
                        val_lpips_loss += lpips_val

                        for j in range(real_free_np.shape[0]):
                            val_psnr_mask += psnr(real_free_np[j], pred_free_np[j], data_range=1)
                            val_ssim_loss += ssim(real_free_np[j], pred_free_np[j], data_range=1, channel_axis=-1)
                            # val_psnr_y += psnr_y(real_free_np[j], pred_free_np[j])
                            # val_ssim_crop4_loss += ssim_crop4(real_free_np[j], pred_free_np[j])

                        pbar_val.update(1)

                        # m += 1

                # Averaging over the entire validation set
                # val_g_loss /= len(val_loader)
                val_l1_loss /= len(val_loader)
                val_ssim_loss /= len(val_loader.dataset)
                # val_ssim_crop4_loss /= len(val_loader.dataset)
                val_lpips_loss /= len(val_loader)
                val_psnr_mask /= len(val_loader.dataset)
                # val_psnr_y /= len(val_loader.dataset)

                print(f"Validation after Epoch [{epoch + 1}/{NUM_EPOCHS}] "
                    f"Val G Loss: 0, Val L1 Loss: {val_l1_loss:.4f}, "
                    f"PSNR_MASK: {val_psnr_mask:.4f}, PSNR_Y: {val_psnr_y:.4f}, "
                    f"SSIM: {val_ssim_loss:.4f}, SSIM_crop4: {val_ssim_crop4_loss:.4f}, "
                    f"LPIPS: {val_lpips_loss:.4f},"
                    f"epoch{e}")
                e = e+5

In [ ]:
# Validation code
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from lpips import LPIPS
import torchvision.models as models

# Constants
LEARNING_RATE = 0.001
NUM_EPOCHS = 500
SAVE_INTERVAL = 5
START_EPOCH = 0  # Start from epoch 80
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#G1.load_state_dict(torch.load(r"C:\Users\nasef\Desktop\SBSR\Models\SBSR32\SBSR30.pth"))


optimizer_G = optim.Adam(G1.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))

# Add the cosine annealing scheduler
scheduler_G = optim.lr_scheduler.CosineAnnealingLR(optimizer_G, T_max=NUM_EPOCHS - START_EPOCH, eta_min=0.000001)

# Loss functions
l1_loss = nn.L1Loss()
mse_loss = nn.MSELoss()
lpips_loss_fn = LPIPS(net='alex').to(DEVICE)  # Using AlexNet for LPIPS loss

# Define the VGG loss
class VGGLoss(nn.Module):
    def __init__(self, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super(VGGLoss, self).__init__()
        vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features.to(device).eval()  # Move VGG layers to device and set to eval mode
        self.vgg_layers = nn.Sequential(*list(vgg)[:36]).to(device)
        self.device = device
        for param in self.vgg_layers.parameters():
            param.requires_grad = False

    def forward(self, x, y):
        x = x.to(self.device)
        y = y.to(self.device)
        x_vgg = self.vgg_layers(x)
        y_vgg = self.vgg_layers(y)
        loss = nn.functional.l1_loss(x_vgg, y_vgg)
        return loss

# Define Focal Frequency Loss
class FocalFrequencyLoss(nn.Module):
    def __init__(self, alpha=1.0, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super(FocalFrequencyLoss, self).__init__()
        self.alpha = alpha
        self.device = device

    def forward(self, input, target):
        input = input.to(self.device)
        target = target.to(self.device)
        input_fft = torch.fft.fft2(input)
        target_fft = torch.fft.fft2(target)
        diff = input_fft - target_fft
        abs_diff = torch.abs(diff)
        loss = torch.pow(abs_diff, self.alpha)
        return torch.mean(loss)

# Loss weights
lambda_vgg = 0.01
lambda_ff = 0.1

# Combined loss
class CombinedLoss(nn.Module):
    def __init__(self, lambda_vgg, lambda_ff, device=DEVICE):
        super(CombinedLoss, self).__init__()
        self.lambda_vgg = lambda_vgg
        self.lambda_ff = lambda_ff
        self.vgg_loss = VGGLoss(device=device)
        self.ff_loss = FocalFrequencyLoss(device=device)

    def forward(self, input, target):
        l1 = l1_loss(input, target)
        vgg = self.vgg_loss(input, target)
        ff = self.ff_loss(input, target)
        return l1 + self.lambda_vgg * vgg + self.lambda_ff * ff

# Example usage
combined_loss = CombinedLoss(lambda_vgg=lambda_vgg, lambda_ff=lambda_ff, device=DEVICE)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lpips_loss_fn = LPIPS(net='alex').to(DEVICE)  # Using AlexNet for LPIPS loss
G1 = RAMiT_FFAT(target_mode='light_realdn',img_norm=False, mv_ver = 3)
# Validation and Checkpoints
G1.eval()
e = 5
while e <= 500:
    G1 = RAMiT_FFAT(target_mode='light_realdn',img_norm=False, mv_ver = 3)
    print(e)
    from collections import OrderedDict
    G1 = G1.to('cuda')
    # Load checkpoint and create new state_dict with 'module.' prefix if needed
    checkpoint = torch.load(f'C:/Users/Admin/Desktop/AI_RAMiT/FFAT_RAMiT/SBSR{e}.pth', map_location='cuda')
    state_dict = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint
    new_state_dict = OrderedDict()

    # Add prefix 'module.' if needed
    new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

    # Apply DataParallel and move to GPU
    G1.load_state_dict(new_state_dict)
    G1 = torch.nn.DataParallel(G1)
    with torch.no_grad():
        val_g_loss = 0.0
        val_l1_loss = 0.0
        val_ssim_loss = 0.0
        val_lpips_loss = 0.0
        val_psnr_mask = 0.0
        m = 0

        with tqdm(total=len(val_loader), desc=f'Validation', unit='batch') as pbar_val:
            for i, (real_shadow, real_free) in enumerate(val_loader):
                real_shadow = real_shadow.to(DEVICE)
                real_free = real_free.to(DEVICE)

                pred_free = G1(real_shadow)
                # Generator loss
                g1_loss = l1_loss(pred_free, real_free)
                g_loss = g1_loss

                val_g_loss += g_loss.item()
                val_l1_loss += g1_loss.item()

                # Calculate SSIM and LPIPS
                pred_free_np = pred_free.cpu().numpy().transpose(0, 2, 3, 1)
                real_free_np = real_free.cpu().numpy().transpose(0, 2, 3, 1)

                lpips_val = lpips_loss_fn(pred_free, real_free).mean().item()  # Average LPIPS over batch
                val_lpips_loss += lpips_val

                # Calculate PSNR for final shadow-free images
                for j in range(real_free_np.shape[0]):
                    val_psnr_mask += psnr(real_free_np[j], pred_free_np[j], data_range=1)
                    val_ssim_loss += ssim(real_free_np[j], pred_free_np[j], data_range=1, channel_axis=-1)

                pbar_val.update(1)
                # save_image(pred_free, os.path.join(sample_dir, f'pred_free_epoch_{m}.png'))
                m += 1

        # Averaging over the entire validation set
        val_g_loss /= len(val_loader)
        val_l1_loss /= len(val_loader)
        val_ssim_loss /= len(val_loader.dataset)
        val_lpips_loss /= len(val_loader)
        val_psnr_mask /= len(val_loader.dataset)

        print(f"Validation "
            f"Val G Loss: {val_g_loss:.4f}, Val L1 Loss: {val_l1_loss:.4f}, "
            f"PSNR_MASK: {val_psnr_mask:.4f}, SSIM: {val_ssim_loss:.4f}, LPIPS: {val_lpips_loss:.4f}")
        e = e+5


In [ ]:
# from google.colab import runtime
# runtime.unassign()

In [ ]:
# top_output, top_inputs, top_targets, low_output, low_inputs, low_targets, tev_out, tev_inputs, tev_targets, output_l, targets_l, inputs_l = train(model, train_loader, 10)
# # val_loss, e_out, total_psnr = evaluate(model, val_loader)
# # val_ppl = math.exp(val_loss)
# elapsed = time.time()
# print('-' * 89)
# print(f'time: {elapsed:5.2f}s ')#| psnr{total_psnr:5.2f} |'
#             # f'valid loss {val_loss:5.2f}')
# print('-' * 89)

# import numpy as np

# # scheduler.step()
#  # load best model states

In [ ]:
# import torch
# from torchvision import transforms
# import matplotlib.pyplot as plt
# from PIL import Image

# # Assuming 'tensor' is your input tensor with shape [2, 3, 256, 256]
# tensor_list = top_output  # Example tensor, replace with your actual tensor
# ind = 0
# # Denormalize the tensor
# for tensor in tensor_list:
#   tensor = tensor * 255  # Scale the tensor values to [0, 255]
#   tensor = tensor.clamp(0, 255)  # Ensure the values are within [0, 255]
#   tensor = tensor.byte()  # Convert to byte type
#   tensor_list[ind] = tensor
#   ind += 1

# # Convert the tensor to a PIL image and display both images

# to_pil_image = transforms.ToPILImage()

# for tensor in tensor_list:
#   for i in range(tensor.shape[0]):
#     image = to_pil_image(tensor[0])
#     plt.imshow(image)
#     plt.axis('off')  # Hide the axes
#     plt.show()
# # for i in range(tensor.shape[0]):
# #     image = to_pil_image(tensor[i])
# #     plt.imshow(image)
# #     plt.axis('off')  # Hide the axes
# #     plt.show()

In [ ]:
# tensor = top_inputs  # Example tensor, replace with your actual tensor

# # Denormalize the tensor
# tensor = tensor * 255  # Scale the tensor values to [0, 255]
# tensor = tensor.clamp(0, 255)  # Ensure the values are within [0, 255]
# tensor = tensor.byte()  # Convert to byte type

# # Convert the tensor to a PIL image and display both images
# to_pil_image = transforms.ToPILImage()

# for i in range(tensor.shape[0]):
#     image = to_pil_image(tensor[i])
#     plt.imshow(image)
#     plt.axis('off')  # Hide the axes
#     plt.show()

In [ ]:
# tensor = top_targets  # Example tensor, replace with your actual tensor

# # Denormalize the tensor
# tensor = tensor * 255  # Scale the tensor values to [0, 255]
# tensor = tensor.clamp(0, 255)  # Ensure the values are within [0, 255]
# tensor = tensor.byte()  # Convert to byte type

# # Convert the tensor to a PIL image and display both images
# to_pil_image = transforms.ToPILImage()

# for i in range(tensor.shape[0]):
#     image = to_pil_image(tensor[i])
#     plt.imshow(image)
#     plt.axis('off')  # Hide the axes
#     plt.show()

In [ ]:
# # import torch
# # from torchvision import transforms
# # import matplotlib.pyplot as plt
# # from PIL import Image

# # Assuming 'tensor' is your input tensor with shape [1, 3, 256, 256]
# # tensor = e_out  # Example tensor, replace with your actual tensor

# # # Remove the batch dimension
# # tensor = tensor.squeeze(0)  # Now the shape is [3, 256, 256]

# # # Convert the tensor to a PIL image
# # to_pil_image = transforms.ToPILImage()
# # image = to_pil_image(tensor)

# # # Display the image
# # plt.imshow(image)
# # plt.axis('off')  # Hide the axes
# # plt.show()


# import torch
# from torchvision import transforms
# import matplotlib.pyplot as plt
# from PIL import Image

# # Assuming 'tensor' is your input tensor with shape [2, 3, 256, 256]
# tensor = e_out  # Example tensor, replace with your actual tensor

# # Denormalize the tensor
# tensor = tensor * 255  # Scale the tensor values to [0, 255]
# tensor = tensor.clamp(0, 255)  # Ensure the values are within [0, 255]
# tensor = tensor.byte()  # Convert to byte type

# # Convert the tensor to a PIL image and display both images
# to_pil_image = transforms.ToPILImage()

# for i in range(tensor.shape[0]):
#     image = to_pil_image(tensor[0])
#     plt.imshow(image)
#     plt.axis('off')  # Hide the axes
#     plt.show()
